<a href="https://colab.research.google.com/github/AIkikr/code-translation-pipeline/blob/main/gpt_oss_SFT_data_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 0. File upload & Drive mount (run first in Colab)
# ---------------------------------------------------------
from google.colab import files, drive, userdata
import os, shutil, re, json, glob, textwrap, random

In [2]:
from google.colab import drive
drive.mount("/content/mydrive", force_remount=True)

from pathlib import Path


# 既存のPDF置き場を再利用
PDF_DIR   = "/content/mydrive/MyDrive/phi4_proj_23/std2_pdfs"   # ←ここを使う
TEXT_DIR  = "/content/mydrive/MyDrive/phi4_proj_23/std2_txt_2"    # ←既存TXT用ディレクトリ（なければ作成）
CHUNK_DIR = "/content/mydrive/MyDrive/phi4_proj_23/chunks_v4"   # ←新規: チャンク出力

os.makedirs(TEXT_DIR, exist_ok=True)
os.makedirs(CHUNK_DIR, exist_ok=True)

pdfs = sorted([str(p) for p in Path(PDF_DIR).glob("*.pdf")])
print("PDF count:", len(pdfs))
assert pdfs, f"No PDFs in {PDF_DIR}"


Mounted at /content/mydrive
PDF count: 315


In [3]:
# ---------------------------------------------------------
# 2. Environment & HF token
# ---------------------------------------------------------
from huggingface_hub import login, list_repo_files, hf_hub_download, upload_folder, create_repo

hf_token = userdata.get("HF_TOKEN")
os.environ["HUGGINGFACE_HUB_TOKEN"] = hf_token or ""
if not hf_token:
    from getpass import getpass
    hf_token = getpass("🔑 Enter your Hugging Face token: ")
    os.environ["HUGGINGFACE_HUB_TOKEN"] = hf_token
!huggingface-cli login --token $HUGGINGFACE_HUB_TOKEN --add-to-git-credential

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
Token is valid (permission: fineGrained).
The token `HF_TOKEN` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `HF_TOKEN`


In [4]:
from IPython.display import Javascript
#JavaScriptコードを埋め込む
keep_alive_code = """
function keepColabAlive() {
    setInterval(() => {
        document.querySelector("colab-toolbar-button#connect").click();
    }, 600000); // 10分ごとにクリック
}
keepColabAlive();
"""

# JavaScriptを実行
display(Javascript(keep_alive_code))

<IPython.core.display.Javascript object>

In [5]:
!ls -R /content/mydrive/MyDrive/phi4_proj_23 | head -n 30


/content/mydrive/MyDrive/phi4_proj_23:
chunks_v1
chunks_v2
chunks_v4
corpus_dual.jsonl
sft_build
std2_pdfs
std2_txt
std2_txt_2
std2_txt_v2

/content/mydrive/MyDrive/phi4_proj_23/chunks_v1:

/content/mydrive/MyDrive/phi4_proj_23/chunks_v2:
03_05_fiÁ‘W.qxd__00000.txt
03_05_ﬁÁ‘W.qxd__00000.txt
03_05_fiÁ‘W.qxd__00001.txt
03_05_ﬁÁ‘W.qxd__00001.txt
03_05_fiÁ‘W.qxd__00002.txt
03_05_ﬁÁ‘W.qxd__00002.txt
03_05_fiÁ‘W.qxd__00003.txt
03_05_ﬁÁ‘W.qxd__00003.txt
03_05_fiÁ‘W.qxd__00004.txt
03_05_ﬁÁ‘W.qxd__00004.txt
03_05_fiÁ‘W.qxd__00005.txt
03_05_ﬁÁ‘W.qxd__00005.txt
03_05_fiÁ‘W.qxd__00006.txt
03_05_ﬁÁ‘W.qxd__00006.txt
04_techdata__00000.txt
04_techdata__00001.txt


In [6]:
!pip install --no-cache-dir "protobuf>=5.29.2,<6.0" "datasets>=3.4,<4.0"
!pip install --no-cache-dir --no-deps "unsloth==2025.8.6" "unsloth_zoo==2025.8.5"
#!pip install --no-cache-dir "transformers==4.55.2" "accelerate==1.10.0" peft bitsandbytes trl


!pip install --quiet "pymupdf>=1.24.9" "pdfminer.six>=20240706" "datasets>=3.4,<4.0" "transformers==4.55.2" "accelerate==1.10.0" peft bitsandbytes "trl>=0.11.4" unsloth

import os, json, re, uuid, pathlib, hashlib, textwrap, random
from pathlib import Path
import fitz  # PyMuPDF
from pdfminer.high_level import extract_text as pdfminer_extract_text
from datasets import Dataset
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 35.0 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.7/182.7 kB 341.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 128.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import google.protobuf as gp
import transformers, accelerate, datasets, unsloth
print("protobuf:", gp.__version__)
print("transformers:", transformers.__version__)
print("accelerate:", accelerate.__version__)
print("datasets:", datasets.__version__)
print("unsloth:", unsloth.__version__)


protobuf: 5.29.5
transformers: 4.55.2
accelerate: 1.10.0
datasets: 3.6.0
unsloth: 2025.8.6


In [8]:
!pip show protobuf transformers accelerate datasets unsloth


Name: protobuf
Version: 5.29.5
Summary: 
Home-page: https://developers.google.com/protocol-buffers/
Author: protobuf@googlegroups.com
Author-email: protobuf@googlegroups.com
License: 3-Clause BSD License
Location: /usr/local/lib/python3.12/dist-packages
Requires: 
Required-by: google-ai-generativelanguage, google-api-core, google-cloud-aiplatform, google-cloud-appengine-logging, google-cloud-audit-log, google-cloud-bigquery-connection, google-cloud-bigquery-storage, google-cloud-bigtable, google-cloud-dataproc, google-cloud-datastore, google-cloud-firestore, google-cloud-functions, google-cloud-language, google-cloud-logging, google-cloud-resource-manager, google-cloud-secret-manager, google-cloud-spanner, google-cloud-speech, google-cloud-trace, google-cloud-translate, google-generativeai, googleapis-common-protos, grpc-google-iam-v1, grpcio-status, kaggle, orbax-checkpoint, proto-plus, tensorboard, tensorflow, tensorflow-datasets, tensorflow-hub, tensorflow-metadata, unsloth, unsloth

In [9]:
# 絶対に旧DIRに上書きしない設定
USE_EXISTING_TEXT = False  # ← 新規抽出に切り替え

PDF_DIR   = "/content/mydrive/MyDrive/phi4_proj_23/std2_pdfs"
WORK_DIR  = "/content/mydrive/MyDrive/phi4_proj_23/sft_build"
OUT_DIR   = f"{WORK_DIR}/out"; os.makedirs(OUT_DIR, exist_ok=True)

TEXT_DIR  = "/content/mydrive/MyDrive/phi4_proj_23/std2_txt_v2"  # ← 新しく作る
CHUNK_DIR = "/content/mydrive/MyDrive/phi4_proj_23/chunks_v2"    # ← 新しい出力先
os.makedirs(TEXT_DIR,  exist_ok=True)
os.makedirs(CHUNK_DIR, exist_ok=True)

# JSONLはクリーンに
Q_FILE    = f"{OUT_DIR}/questions.jsonl"
ANS_FILE  = f"{OUT_DIR}/answers.jsonl"
SFT_FILE  = f"{OUT_DIR}/dataset.jsonl"
for p in [Q_FILE, ANS_FILE, SFT_FILE]:
    if os.path.exists(p): os.remove(p)


In [10]:
# 4) 乱数固定（安定再現）
import numpy as np, random
random.seed(41); np.random.seed(41)
if torch.cuda.is_available():
    torch.manual_seed(41); torch.cuda.manual_seed_all(41)

In [11]:
# ★ transformers より先に必ず unsloth
import unsloth
from unsloth import FastLanguageModel, UnslothTrainer, UnslothTrainingArguments

# 念のため、過去に作ったHF Trainerインスタンスが残っていたら消す
for name in list(globals().keys()):
    if name.lower() == "trainer":
        del globals()[name]

In [12]:
from transformers import DataCollatorForLanguageModeling
import torch, pathlib

# 1) モデル & LoRA（あなたが既に実行した設定と同じ）
MODEL_ID = "unsloth/gpt-oss-20b-unsloth-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
    MODEL_ID, load_in_4bit=True, dtype=None, max_seq_length=4096
)
tokenizer.pad_token = tokenizer.eos_token
model.eval(); model.config.use_cache = True
DEVICE = model.device

==((====))==  Unsloth 2025.8.6: Fast Gpt_Oss patching. Transformers: 4.55.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gpt_Oss does not support SDPA - switching to eager!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [53]:
def gen(messages, max_new_tokens=512, do_sample=False, temperature=0.0, add_generation_prompt=False):
    # 1) chatテンプレ適用 → Tensor or dict どちらでも受ける
    enc = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=add_generation_prompt,
        return_tensors="pt",
    )
    if isinstance(enc, torch.Tensor):
        input_ids = enc
        attention_mask = torch.ones_like(input_ids)  # ★ 必ず付与
    else:
        input_ids = enc["input_ids"]
        attention_mask = enc.get("attention_mask")
        if attention_mask is None:
            attention_mask = torch.ones_like(input_ids)  # ★ 必ず付与

    # 2) デバイスへ
    input_ids = input_ids.to(DEVICE)
    attention_mask = attention_mask.to(DEVICE)

    # 3) 生成
    with torch.no_grad():
        out = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,  # ★ 重要
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    new_tokens = out[0, input_ids.shape[1]:]
    return tokenizer.decode(new_tokens, skip_special_tokens=True).strip()


In [54]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
if getattr(model.config, "pad_token_id", None) is None:
    model.config.pad_token_id = tokenizer.eos_token_id


In [14]:
# ---- PDF→テキスト（必要時のみ） ----
import fitz
from pdfminer.high_level import extract_text as pdfminer_extract_text

def extract_text_pymupdf(pdf_path):
    doc = fitz.open(pdf_path)
    meta = doc.metadata or {}
    texts = []
    for page in doc:
        texts.append(page.get_text("text"))
    doc.close()
    return "\n".join(texts), meta

def extract_text_fallback(pdf_path):
    try:
        return pdfminer_extract_text(pdf_path) or "", {}
    except Exception:
        return "", {}

import unicodedata, re
from pathlib import Path

def safe_stem(path_like):
    raw = Path(path_like).stem
    s = unicodedata.normalize("NFKC", raw)            # 全角→半角など正規化
    s = re.sub(r'[\/\\:\*\?"<>\|]', "_", s)           # 禁止文字を _
    s = re.sub(r"[\u0000-\u001F]", "_", s)            # 制御文字を _
    s = s.strip(" .")                                 # 末尾の空白/ドット除去
    if len(s) > 150: s = s[:150]                      # 長すぎ防止
    return s or "untitled"


def save_text(pdf_path):
    name = safe_stem(pdf_path)
    out_txt  = f"{TEXT_DIR}/{name}.txt"
    out_meta = f"{TEXT_DIR}/{name}.meta.json"
    if os.path.exists(out_txt) and os.path.exists(out_meta):
        return out_txt

    txt, meta = extract_text_pymupdf(pdf_path)
    if len((txt or "").strip()) < 50:
        txt, meta = extract_text_fallback(pdf_path)

    # ▼ ここを追加：抽出テキストを正規化
    txt = j_normalize(txt)
    with open(out_txt, "w", encoding="utf-8", errors="ignore") as f:
        f.write(txt or "")
    with open(out_meta, "w", encoding="utf-8") as f:
        json.dump({"pdf": pdf_path, "meta": meta or {}}, f, ensure_ascii=False)
    return out_txt


In [15]:
# === Normalisation utilities (最上部：import直後に追加) ===
import unicodedata, re

def j_normalize(s: str) -> str:
    if not s: return ""
    # NFKC & 結合分音(Mn)除去
    s = unicodedata.normalize("NFKC", s)
    s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")
    # カタカナの不自然な空白を除去（イ ー サ ネ ッ ト → イーサネット）
    s = re.sub(r"([ァ-ヶｦ-ﾟ])\s+(?=[ァ-ヶｦ-ﾟ])", "", s)
    # 異常な不可視空白の圧縮
    s = re.sub(r"[\u2000-\u200F\u2028\u2029]", " ", s)
    return s

In [ ]:
pdfs = sorted([str(p) for p in Path(PDF_DIR).glob("*.pdf")])
assert pdfs, f"No PDFs in {PDF_DIR}"

for p in pdfs:
    try:
        _ = save_text(p)      # サニタイズ済み save_text で保存
    except Exception as e:
        print("FAILED:", p, e)

FAILED: /content/mydrive/MyDrive/phi4_proj_23/std2_pdfs/HOTnet、Infineraのオープン・オプティカル・ネットワーキング・ソリューションを導入し、長距離トランスポート・ネットワークを強化 _ インフィネラ・ジャパン株式会社のプレスリリース.pdf [Errno 22] Invalid argument: '/content/mydrive/MyDrive/phi4_proj_23/std2_txt_v2/HOTnet、Infineraのオープン・オプティカル・ネットワーキング・ソリューションを導入し、長距離トランスポート・ネットワークを強化 _ インフィネラ・ジャパン株式会社のプレスリリース.meta.json'


In [16]:
META_RE_YEAR   = re.compile(r"(19|20)\d{2}")
META_RE_AUTHOR = re.compile(r"(?i)\bby\s+([A-Z][\w\.\- ]{2,100})")

def basic_guess(txt, pdfmeta):
    head = (txt or "")[:2000]
    lines = [l.strip() for l in head.splitlines() if l.strip()]
    # PDFメタtitleがあれば最優先
    meta_title = (pdfmeta.get("title") or "").strip() if isinstance(pdfmeta, dict) else ""
    cand_title = meta_title if len(meta_title) >= 5 else (lines[0] if lines else "Unknown")
    if len(cand_title) < 5 and len(lines) > 1:
        cand_title = lines[1]
    m = META_RE_AUTHOR.search(head)
    author = m.group(1).strip() if m else (pdfmeta.get("author") or "Unknown")
    y = META_RE_YEAR.search(head) or META_RE_YEAR.search((pdfmeta.get("creationDate") or ""))
    year = y.group(0) if y else "Unknown"
    return cand_title[:200], str(author)[:120], str(year)

def _pick_header_line(text):
    m = re.search(r"【[^【】\n]+】", text)
    return m.group(0) if m else ""

def _is_placeholder(header):
    s = header.replace("｜", "/").lower()
    return ("title" in s) or ("author" in s) or ("year" in s) or (header.strip() == "")

def extract_meta_candidates(txt, rough):
    sys = "あなたは学術記事のメタデータを確認するアシスタントです。"
    user = (
        "以下の本文冒頭から、記事タイトル、著者、出版年の候補をできるだけ挙げてください。\n"
        "わからない場合も『Unknown』と書いてください。\n"
        "---\n" + (txt or "")[:2000] + "\n---\n"
        f"粗推定: title='{rough[0]}', author='{rough[1]}', year='{rough[2]}'"
    )
    out = gen([{"role":"system","content":sys},{"role":"user","content":user}],
              max_new_tokens=128, do_sample=False)
    return out

def strict_extract_with_llm(txt, rough):
    # 1st pass: 候補抽出
    candidates = extract_meta_candidates(txt, rough)

    # 2nd pass: 整形（必ず1行、ラベル無し）
    sys = ("あなたは学術メタデータ抽出器です。返答は必ず1行のみ。"
           "形式は【<タイトル>｜<著者>｜<年>】。"
           "未知は Unknown。余計な文を付けない。")
    user = (
        "以下の候補を点検し、最も適切と思われるタイトル/著者/出版年を1つずつ選び、"
        "フォーマットにまとめてください。\n"
        "候補:\n" + candidates
    )
    out = gen([{"role":"system","content":sys},{"role":"user","content":user}],
              max_new_tokens=64, do_sample=False)
    m = re.search(r"【[^【】\n]+｜[^【】\n]+｜[^【】\n]+】", out)
    if m:
        return m.group(0)

    # 最後のフォールバック（必ず値で返す）
    return f""

def build_meta_for_textfile(txt_path):
    name = Path(txt_path).stem
    txt  = open(txt_path, encoding="utf-8", errors="ignore").read()
    pdfmeta = {}
    meta_json = Path(f"{TEXT_DIR}/{name}.meta.json")
    if meta_json.exists():
        try:
            pdfmeta = json.load(open(meta_json, encoding="utf-8"))["meta"]
        except Exception:
            pdfmeta = {}
    rough = basic_guess(txt, pdfmeta)
    return strict_extract_with_llm(txt, rough)  # "【Title｜Author｜Year】"

# ---- 25%オーバーラップのチャンク化 ----
def split_by_tokens(text, max_tokens=1024, overlap_ratio=0.25):
    ids = tokenizer(text).input_ids
    chunks = []
    stride = max(1, int(max_tokens * (1 - overlap_ratio)))  # 1024 * 0.75 = 768
    i = 0
    while i < len(ids):
        sub = ids[i:i+max_tokens]
        if len(sub) < int(max_tokens * 0.2):
            break
        chunks.append(tokenizer.decode(sub, skip_special_tokens=True))
        if i + max_tokens >= len(ids):
            break
        i += stride
    return chunks

def make_chunks(txt_path):
    name  = Path(txt_path).stem
    meta  = build_meta_for_textfile(txt_path)    # "【Title｜Author｜Year】"
    src   = open(txt_path, encoding="utf-8", errors="ignore").read()
    bodies = split_by_tokens(src, max_tokens=1024, overlap_ratio=0.25)
    out_files = []
    for idx, body in enumerate(bodies):
        chunk_id = f"{name}__{idx:05d}"
        payload = f"{meta}\n{body.strip()}"
        outp = f"{CHUNK_DIR}/{chunk_id}.txt"
        with open(outp,"w",encoding="utf-8") as f:
            f.write(payload)
        out_files.append(outp)
    return out_files


In [17]:
def get_meta_header(txt_path):
    """絶対に空を返さないヘッダ生成。空や不正なら rough で埋める。"""
    name = Path(txt_path).stem
    txt  = open(txt_path, encoding="utf-8", errors="ignore").read()
    pdfmeta = {}
    meta_json = Path(f"{TEXT_DIR}/{name}.meta.json")
    if meta_json.exists():
        try:
            pdfmeta = json.load(open(meta_json, encoding="utf-8"))["meta"]
        except Exception:
            pdfmeta = {}

    rough = basic_guess(txt, pdfmeta)  # (title, author, year)
    # 既存のLLM抽出（2段階版）を呼ぶ
    meta = strict_extract_with_llm(txt, rough)

    # ---- フェイルセーフ（ここが肝）----
    # 空/None/不正フォーマットなら rough で必ず埋める
    if not meta or "【" not in meta or "】" not in meta or "｜" not in meta:
        if isinstance(rough, (tuple, list)) and len(rough) >= 3:
            rtitle, rauthor, ryear = rough[0], rough[1], rough[2]
        elif isinstance(rough, dict):
            rtitle, rauthor, ryear = rough.get("title","Unknown"), rough.get("author","Unknown"), rough.get("year","Unknown")
        else:
            rtitle = rauthor = ryear = "Unknown"
        meta = f"【{rtitle}｜{rauthor}｜{ryear}】"
    return meta

In [18]:
# === FIXED gen(): mapping で渡す、安全ガード付き ===
def gen(messages, max_new_tokens=512, do_sample=False, temperature=0.0,
        add_generation_prompt=False):
    # 1) chatテンプレ適用 → Tensor/BatchEncoding どちらでも受ける
    enc = tokenizer.apply_chat_template(
        messages,
        tokenize=True,                 # ★ 重要: dict を返させる
        add_generation_prompt=add_generation_prompt,
        return_tensors="pt",
    )
    # 2) 万一 Tensor だったらラップ
    if isinstance(enc, torch.Tensor):
        enc = {"input_ids": enc}
    # 3) GPUへ
    enc = {k: v.to(DEVICE) for k, v in enc.items()}

    with torch.no_grad():
        out = model.generate(
            **enc,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    # 入力ぶんをカットして新規生成だけデコード
    new_tokens = out[0, enc["input_ids"].shape[1]:]
    return tokenizer.decode(new_tokens, skip_special_tokens=True).strip()


In [ ]:
## 実行（既存TEXTの再利用前提）
#chunk_files = []
#for t in sorted(Path(TEXT_DIR).glob("*.txt")):
#    if t.name.endswith(".meta.json"):
#        continue
#    chunk_files += make_chunks(str(t))
#print("chunks:", len(chunk_files), "→ 例:", chunk_files[0] if chunk_files else "N/A")


In [19]:
import unicodedata, re, hashlib

# 既存チャンク→元TXTを引けるようにするマップ（重複作成防止にも使う）
CHUNK_MAP_PATH = Path(CHUNK_DIR) / ".map.jsonl"
CHUNK_MAP_PATH.parent.mkdir(parents=True, exist_ok=True)

def _safe_base(name: str, max_len: int = 120) -> str:
    s = unicodedata.normalize("NFKC", name)
    s = re.sub(r'[\/\\:\*\?"<>\|\n\r\t]', "_", s)  # 禁止文字
    s = s.strip(" .")                              # 末尾スペース/ドット
    return s[:max_len] if len(s) > max_len else (s or "untitled")

def _load_existing_pairs():
    pairs = set()  # (src_txt_abs, idx)
    if CHUNK_MAP_PATH.exists():
        with open(CHUNK_MAP_PATH, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    d = json.loads(line)
                    pairs.add((d["src_txt"], int(d["idx"])))
                except Exception:
                    pass
    return pairs

def _record_map(chunk_path: str, src_txt: str, idx: int, orig_base: str):
    rec = {"chunk": chunk_path, "src_txt": src_txt, "idx": idx, "orig_base": orig_base}
    with open(CHUNK_MAP_PATH, "a", encoding="utf-8") as mf:
        mf.write(json.dumps(rec, ensure_ascii=False) + "\n")

def _write_chunk_file(orig_base: str, idx: int, content: str):
    # 長い名前→短縮→さらに短縮→最終手段はハッシュ
    candidates = [
        _safe_base(orig_base, 120),
        _safe_base(orig_base, 90),
        _safe_base(orig_base, 60),
        hashlib.md5(orig_base.encode("utf-8")).hexdigest()[:16],
    ]
    last_exc = None
    for base in candidates:
        outp = f"{CHUNK_DIR}/{base}__{idx:05d}.txt"
        try:
            with open(outp, "w", encoding="utf-8") as f:
                f.write(content)
            return outp  # 成功
        except OSError as e:
            last_exc = e
            continue
    raise last_exc


In [20]:
# --- Add: safe marker filename (avoid long/illegal names) ---
import hashlib, unicodedata, re
from pathlib import Path

def _safe_marker_path(stem: str) -> Path:
    # 既に _safe_base があるなら使う。無ければ簡易版で代用。
    try:
        base = _safe_base(stem, 80)  # 前のパッチで定義済みのはず
    except NameError:
        s = unicodedata.normalize("NFKC", stem)
        s = re.sub(r'[\/\\:\*\?"<>\|\n\r\t]', "_", s).strip(" .")
        base = (s[:80] if len(s) > 80 else (s or "untitled"))
    h = hashlib.md5(stem.encode("utf-8")).hexdigest()[:8]
    return Path(DONE_DIR) / f"{base}.{h}.done"


In [21]:
from pathlib import Path
import os, json, sys, time

# 再開用: 既に完了したテキストをスキップ
DONE_DIR = f"{CHUNK_DIR}/.done"
os.makedirs(DONE_DIR, exist_ok=True)

def make_chunks_idempotent(txt_path):
    orig_base = Path(txt_path).stem  # 元TXTの見た目名前（長くてもOK）
    meta  = get_meta_header(txt_path)  # 【Title｜Author｜Year】（必ず非空）
    src   = open(txt_path, encoding="utf-8", errors="ignore").read()
    bodies = split_by_tokens(src, max_tokens=1024, overlap_ratio=0.25)

    # 既に作った (src_txt, idx) を読み込み（重複出力ガード）
    existing_pairs = _load_existing_pairs()

    made = 0
    for idx, body in enumerate(bodies):
        pair_key = (str(Path(txt_path).resolve()), idx)
        if pair_key in existing_pairs:
            continue  # すでにマップに記録済み（どこかの名前で出力済み）

        # まずはデフォルト安全名で存在確認（早期スキップ）
        default_safe = _safe_base(orig_base, 120)
        default_path = f"{CHUNK_DIR}/{default_safe}__{idx:05d}.txt"
        if os.path.exists(default_path):
            _record_map(default_path, str(Path(txt_path).resolve()), idx, orig_base)
            made += 0
            continue

        payload = f"{meta}\n{body.strip()}"
        try:
            written = _write_chunk_file(orig_base, idx, payload)
            _record_map(written, str(Path(txt_path).resolve()), idx, orig_base)
            made += 1
        except OSError as e:
            # 書けなかった場合はログだけ吐いて次へ（全体は止めない）
            print(f"[WARN] failed to write chunk idx={idx} for {orig_base}: {e}")
            sys.stdout.flush()
            continue

    return made, len(bodies)





In [22]:
count_total = 0
count_new   = 0
t0 = time.time()

txt_list = [p for p in sorted(Path(TEXT_DIR).glob("*.txt")) if not p.name.endswith(".meta.json")]

for i, tpath in enumerate(txt_list, 1):
    stem = tpath.stem
    marker = _safe_marker_path(stem)
# 旧方式の長いマーカー（互換のため見るだけ）
    legacy_marker = Path(DONE_DIR) / f"{stem}.done"
    if marker.exists() or legacy_marker.exists():
        # 進捗ログ（軽く）
        if i % 100 == 0:
            print(f"[skip:{i}/{len(txt_list)}] {stem} (already chunked)")
        continue

    new_made, total_needed = make_chunks_idempotent(str(tpath))
    count_total += total_needed
    count_new   += new_made

    # このテキストのチャンク化完了マーク（原子書き込み）
    tmp = str(marker) + ".tmp"
    with open(tmp, "w", encoding="utf-8") as f:
      f.write(f"{new_made}/{total_needed}")
    os.replace(tmp, marker)

    # 100ファイルごとに進捗
    if i % 100 == 0 or i == len(txt_list):
        dt = time.time() - t0
        print(f"[progress] {i}/{len(txt_list)} texts; new_chunks={count_new}, total_planned={count_total}, elapsed={dt:.1f}s")
        sys.stdout.flush()

print("✅ chunking done")


[skip:100/323] L1_L2伝送装置(Ethernet_OTN)の2022年_2025年5月までの技術動向と運用変更レポート (already chunked)
[skip:200/323] What is ALS function_ - Knowledge (already chunked)
[skip:300/323] rfc8424 (already chunked)
✅ chunking done


In [23]:
# === safe meta filename + fallback store ===
import hashlib, unicodedata, re
from pathlib import Path

META_STORE = Path(TEXT_DIR) / ".meta_store"
META_STORE.mkdir(exist_ok=True)

def _safe_base(name: str, max_len: int = 120):
    s = unicodedata.normalize("NFKC", name)
    s = re.sub(r'[\/\\:\*\?"<>\|\n\r\t]', "_", s)  # 禁止文字
    s = s.strip(" .")
    return s[:max_len] if len(s) > max_len else (s or "untitled")

def _meta_paths_for(stem: str):
    # 1) 同ディレクトリに安全名で置く候補
    safe = _safe_base(stem, 110)
    primary = Path(TEXT_DIR) / f"{safe}.meta.json"
    # 2) 長すぎ/NGなら退避先（ハッシュ付）
    h = hashlib.md5(stem.encode("utf-8")).hexdigest()[:16]
    alt = META_STORE / f"{_safe_base(stem, 60)}.{h}.meta.json"
    return primary, alt

def save_meta_for_text(txt_path: str, obj: dict) -> str:
    stem = Path(txt_path).stem
    primary, alt = _meta_paths_for(stem)
    try:
        with open(primary, "w", encoding="utf-8") as f:
            json.dump(obj or {}, f, ensure_ascii=False, indent=2)
        return str(primary)
    except OSError:
        pass
    with open(alt, "w", encoding="utf-8") as f:
        json.dump(obj or {}, f, ensure_ascii=False, indent=2)
    return str(alt)

def load_meta_for_text(txt_path: str):
    """優先順: 旧そのまま名 → 安全名 → 退避先"""
    stem = Path(txt_path).stem
    legacy = Path(TEXT_DIR) / f"{stem}.meta.json"
    if legacy.exists():
        try:
            return json.load(open(legacy, encoding="utf-8")), str(legacy)
        except Exception:
            pass
    primary, alt = _meta_paths_for(stem)
    for p in (primary, alt):
        if Path(p).exists():
            try:
                return json.load(open(p, encoding="utf-8")), str(p)
            except Exception:
                pass
    return {}, None


In [24]:
# === Add: TEXT_DIR に必ず .meta.json を作成/強化 ===
import time, unicodedata, re

YEAR_RE_ANY = re.compile(r'\b(19|20)\d{2}\b')

def extract_year_from_meta(meta: dict) -> str:
    if not isinstance(meta, dict): return "Unknown"
    keys = ["CreationDate","ModDate","creationDate","modDate","date","Date","created","modified"]
    for k in keys:
        v = meta.get(k) or meta.get(f"/{k}")
        if not v: continue
        s = unicodedata.normalize("NFKC", str(v))
        m = re.search(r'(?:D:)?((?:19|20)\d{2})', s)
        if m: return m.group(1)
        m2 = YEAR_RE_ANY.search(s)
        if m2: return m2.group(0)
    return "Unknown"

def meta_candidates_from_pdfmeta(meta: dict):
    title  = (meta.get("title")  or meta.get("/Title")  or "").strip()
    author = (meta.get("author") or meta.get("/Author") or "").strip()
    year   = extract_year_from_meta(meta)
    return title, author, year

def ensure_text_meta_json(TEXT_DIR, limit=None):
    YEAR_RE_ANY = re.compile(r'\b(19|20)\d{2}\b')
    def extract_year_from_meta(meta: dict) -> str:
        if not isinstance(meta, dict): return "Unknown"
        keys = ["CreationDate","ModDate","creationDate","modDate","date","Date","created","modified"]
        for k in keys:
            v = meta.get(k) or meta.get(f"/{k}")
            if not v: continue
            s = unicodedata.normalize("NFKC", str(v))
            m = re.search(r'(?:D:)?((?:19|20)\d{2})', s)
            if m: return m.group(1)
            m2 = YEAR_RE_ANY.search(s)
            if m2: return m2.group(0)
        return "Unknown"

    def meta_candidates_from_pdfmeta(meta: dict):
        title  = (meta.get("title")  or meta.get("/Title")  or "").strip()
        author = (meta.get("author") or meta.get("/Author") or "").strip()
        year   = extract_year_from_meta(meta)
        return title, author, year

    done = 0
    for tpath in sorted(Path(TEXT_DIR).glob("*.txt")):
        name = tpath.stem
        # 既存ロード（安全探索）
        existing, _ = load_meta_for_text(str(tpath))
        existing = existing or {}

        # PDFメタ（既存にあれば尊重）
        pdf_meta = (existing.get("meta") or {})
        # 本文先頭から粗推定（未設定時のみ作成）
        txt = open(tpath, encoding="utf-8", errors="ignore").read()
        head = txt[:3000]
        if "rough_guess" not in existing:
            lines = [l.strip() for l in head.splitlines() if l.strip()]
            mtitle, mauthor, myear = meta_candidates_from_pdfmeta(pdf_meta)
            cand_title = mtitle if len(mtitle) >= 5 else (lines[0] if lines else "Unknown")
            if len(cand_title) < 5 and len(lines) > 1:
                cand_title = lines[1]
            m = re.search(r"(?i)\bby\s+([A-Z][\w\.\- ]{2,100})", head)
            cand_author = mauthor if mauthor else (m.group(1).strip() if m else "Unknown")
            y = re.search(r"(19|20)\d{2}", head)
            cand_year = myear if myear != "Unknown" else (y.group(0) if y else "Unknown")
            existing["rough_guess"] = {
                "title": cand_title[:200],
                "author": str(cand_author)[:120],
                "year": str(cand_year),
            }

        existing.setdefault("pdf", existing.get("pdf", ""))
        existing.setdefault("meta", pdf_meta)

        # 安全に保存（同階層→ダメなら .meta_store）
        save_meta_for_text(str(tpath), existing)

        done += 1
        if limit and done >= limit:
            break
    print(f"ensure_text_meta_json: processed {done} files in {TEXT_DIR}")

In [ ]:
# 実行（先に一度走らせるだけでOK。以後チャンク修復で参照）
ensure_text_meta_json(TEXT_DIR)

ensure_text_meta_json: processed 323 files in /content/mydrive/MyDrive/phi4_proj_23/std2_txt_v2


In [25]:
# ========= 追加：chunk → src_txt の辞書を一度だけ読み込む =========
_CHUNK_SRC_CACHE = None
def _load_chunk_map_dict():
    global _CHUNK_SRC_CACHE
    if _CHUNK_SRC_CACHE is not None:
        return _CHUNK_SRC_CACHE
    d = {}
    if CHUNK_MAP_PATH.exists():
        with open(CHUNK_MAP_PATH, encoding="utf-8") as f:
            for line in f:
                try:
                    j = json.loads(line)
                    ch = j.get("chunk"); src = j.get("src_txt")
                    if ch and src:
                        # フルパスでもファイル名でも引けるように両方入れる
                        d[str(ch)] = str(src)
                        d[Path(ch).name] = str(src)
                except Exception:
                    pass
    _CHUNK_SRC_CACHE = d
    return d


# ========= 置換：LLMヘッダー生成（候補→整形→ダメなら rough で埋める） =========
HEADER_OK = re.compile(r"^【[^【】\n]+｜[^【】\n]+｜[^【】\n]+】$")

def llm_header_from_text_and_meta(txt: str, meta_obj: dict, rough):
    # rough はタプルでもdictでもOKにする
    if isinstance(rough, (tuple, list)) and len(rough) >= 3:
        rtitle, rauthor, ryear = rough[0], rough[1], rough[2]
    elif isinstance(rough, dict):
        rtitle, rauthor, ryear = rough.get("title","Unknown"), rough.get("author","Unknown"), rough.get("year","Unknown")
    else:
        rtitle = rauthor = ryear = "Unknown"

    # 1) 候補抽出（本文+PDFメタ）
    sys1  = "あなたは学術記事のメタデータを確認するアシスタントです。"
    user1 = (
        "以下の本文冒頭とPDFメタ情報から、記事タイトル、著者、出版年の候補を挙げてください。\n"
        "本文に無い推測はしない。Unknown可。\n"
        "----[本文冒頭]----\n" + (txt or "")[:2000] + "\n"
        "----[PDFメタ]----\n" + json.dumps(meta_obj or {}, ensure_ascii=False)
    )
    cands = gen([{"role":"system","content":sys1},{"role":"user","content":user1}],
                max_new_tokens=160, do_sample=False)

    # 2) 整形（1行・指定形式）
    sys2  = ("あなたは学術メタデータ抽出器です。返答は必ず1行のみ。"
             "形式は【<タイトル>｜<著者>｜<年>】。未知は Unknown。余計な文は付けない。")
    user2 = ("以下の候補と粗推定を点検し、本文+メタに矛盾しない値を1つずつ選び、"
             "指定形式で出力してください。\n"
             f"【候補】\n{cands}\n"
             f"【粗推定】\n{json.dumps({'title':rtitle,'author':rauthor,'year':ryear}, ensure_ascii=False)}")
    out = gen([{"role":"system","content":sys2},{"role":"user","content":user2}],
              max_new_tokens=64, do_sample=False)
    m = re.search(r"【[^【】\n]+｜[^【】\n]+｜[^【】\n]+】", out)
    if m:
        return m.group(0)

    # 3) 最終フォールバック（必ず値で返す）
    return f""

# ========= 置換：fix 関数（マップ参照→TXT/メタ取得→LLM整形→書き戻し） =========
FIXED_DIR = Path(CHUNK_DIR) / ".fixed"
FIXED_DIR.mkdir(exist_ok=True)

def fix_chunk_headers_llm(CHUNK_DIR, TEXT_DIR, batch=200, dry_run=True):
    """チャンク先頭が不正なものを、.map.jsonl を使って元TXT/メタへ辿り、LLMで修復。
       見つからない場合はチャンク本文から直接推定。処理済みは .fixed マーカーでスキップ。"""
    fixed = 0
    scanned = 0
    cmap = _load_chunk_map_dict()

    for cf in sorted(Path(CHUNK_DIR).glob("*.txt")):
        scanned += 1
        marker = FIXED_DIR / (cf.stem + ".fixed")
        if marker.exists():
            continue

        lines = open(cf, encoding="utf-8").read().splitlines()
        if not lines:
            marker.write_text("skip-empty"); continue
        head = (lines[0] or "").strip()
        if HEADER_OK.match(head):
            marker.write_text("ok"); continue

        # --- 1) マップから元TXTを探す（最優先） ---
        src_txt = cmap.get(str(cf)) or cmap.get(cf.name)
        # --- 2) ダメなら旧ロジック（名前推測） ---
        if not src_txt:
            name_part = cf.stem.split("__")[0]
            guessed = Path(TEXT_DIR) / f"{name_part}.txt"
            if guessed.exists():
                src_txt = str(guessed)

        # --- 3) TXT/メタ/rough を用意 ---
        if src_txt and Path(src_txt).exists():
            txt = open(src_txt, encoding="utf-8", errors="ignore").read()
            meta_obj, _meta_path = load_meta_for_text(src_txt)  # 安全探索（.meta_store も見る）
            pdfmeta = (meta_obj.get("meta") or {})
            rough   = meta_obj.get("rough_guess") or basic_guess(txt, pdfmeta)
        else:
            # 最後の手段：チャンク本文だけで推定（1行目はヘッダ候補なので除外）
            txt = "\n".join(lines[1:])[:4000]
            pdfmeta = {}
            rough   = basic_guess(txt, pdfmeta)

        # --- 4) LLMでヘッダー生成（強フォールバック付き） ---
        new_header = llm_header_from_text_and_meta(txt, pdfmeta, rough)

        if dry_run:
            print(f"[DRY] {cf.name}: '{head}' -> '{new_header}'")
            marker.write_text("dry-run")
        else:
            # 先頭置換 or 先頭に追加
            if head.startswith("【") and "】" in head:
                lines[0] = new_header
            else:
                lines.insert(0, new_header)
            with open(cf, "w", encoding="utf-8") as f:
                f.write("\n".join(lines))
            marker.write_text("fixed")
            fixed += 1

        if fixed >= batch and not dry_run:
            break

    print(f"fix_chunk_headers_llm: scanned={scanned}, fixed={fixed}, dry_run={dry_run}")

In [ ]:
# 先にメタJSONを整備（追加①）
#ensure_text_meta_json(TEXT_DIR)

In [ ]:
# 追いかけ修復（追加②）
fix_chunk_headers_llm(CHUNK_DIR, TEXT_DIR, dry_run=True, batch=50)   # 確認

[DRY] 03_05_fiÁ‘W.qxd__00000.txt: '' -> ''
[DRY] 03_05_fiÁ‘W.qxd__00001.txt: '' -> ''
[DRY] 03_05_fiÁ‘W.qxd__00002.txt: '' -> ''
[DRY] 03_05_fiÁ‘W.qxd__00003.txt: '' -> ''
[DRY] 03_05_fiÁ‘W.qxd__00004.txt: '' -> ''
[DRY] 03_05_fiÁ‘W.qxd__00005.txt: '' -> ''
[DRY] 03_05_fiÁ‘W.qxd__00006.txt: '' -> ''
[DRY] 03_05_ﬁÁ‘W.qxd__00000.txt: '' -> ''
[DRY] 03_05_ﬁÁ‘W.qxd__00001.txt: '' -> ''
[DRY] 03_05_ﬁÁ‘W.qxd__00002.txt: '' -> ''
[DRY] 03_05_ﬁÁ‘W.qxd__00003.txt: '' -> ''
[DRY] 03_05_ﬁÁ‘W.qxd__00004.txt: '' -> ''
[DRY] 03_05_ﬁÁ‘W.qxd__00005.txt: '' -> ''
[DRY] 03_05_ﬁÁ‘W.qxd__00006.txt: '' -> ''
[DRY] 04_techdata__00000.txt: '' -> ''
[DRY] 04_techdata__00001.txt: '' -> ''
[DRY] 04_techdata__00002.txt: '' -> ''
[DRY] 04_techdata__00003.txt: '' -> ''
[DRY] 04_techdata__00004.txt: '' -> ''
[DRY] 04_techdata__00005.txt: '' -> ''
[DRY] 04_techdata__00006.txt: '' -> ''
[DRY] 04_techdata__00007.txt: '' -> ''
[DRY] 04_techdata__00008.txt: '' -> ''
[DRY] 04_techdata__00009.txt: '' -> ''
[

KeyboardInterrupt: 

In [ ]:
fix_chunk_headers_llm(CHUNK_DIR, TEXT_DIR, dry_run=False, batch=200) # 本書換え（数は適宜）

In [26]:
# === タイトルは src_txt のファイル名（stem）をそのまま使う ===
import re, json, os, unicodedata
from pathlib import Path

# すでに定義済みなら再利用
try:
    _CHUNK_SRC_CACHE
except NameError:
    _CHUNK_SRC_CACHE = None

def _load_chunk_map_dict():
    """CHUNK_MAP_PATH(.map.jsonl) から {chunk名 or chunkフルパス → src_txtフルパス} を作る"""
    global _CHUNK_SRC_CACHE
    if _CHUNK_SRC_CACHE is not None:
        return _CHUNK_SRC_CACHE
    d = {}
    if 'CHUNK_MAP_PATH' in globals() and Path(CHUNK_MAP_PATH).exists():
        with open(CHUNK_MAP_PATH, encoding="utf-8") as f:
            for line in f:
                try:
                    j = json.loads(line)
                    ch = j.get("chunk"); src = j.get("src_txt")
                    if ch and src:
                        d[str(ch)] = str(src)
                        d[Path(ch).name] = str(src)
                except Exception:
                    pass
    _CHUNK_SRC_CACHE = d
    return d

# 既存ヘッダー判定（従来の三項フォーマット or 1項フォーマット）
HDR_TRIPLE = re.compile(r'^【[^【】\n]+｜[^【】\n]+｜[^【】\n]+】$')
HDR_SINGLE = re.compile(r'^【[^【】\n]+】$')

def _title_from_src_txt(src_txt_path: str) -> str:
    """ファイル名（stem）をそのままタイトルとして使う。"""
    title = Path(src_txt_path).stem
    # 見栄え微調整: 連続アンダースコア→空白、全角半角正規化など（任意）
    title = unicodedata.normalize("NFKC", title)
    title = re.sub(r'__+', ' ', title)
    title = title.strip()
    return title or "Unknown"

def _new_header(title: str, style="triple") -> str:
    if style == "single":
        return f""
    # 既存パイプライン互換の三項に揃える（著者/年は Unknown）
    return f"【{title}｜Unknown｜Unknown】"

FIXED_DIR = Path(CHUNK_DIR) / ".fixed_title_from_stem"
FIXED_DIR.mkdir(exist_ok=True)

def fix_chunk_headers_from_filename(CHUNK_DIR, TEXT_DIR, style="triple", dry_run=True, batch=200):
    """
    各チャンクの先頭に《src_txt の stem》由来のヘッダーを付与/置換する。
    - .map.jsonl があれば chunk→src_txt をそれで解決（安全名/短縮名でもOK）
    - なければ chunk名の前半（__より前）で TEXT_DIR/{stem}.txt を推測
    - ★既に正しい三項ヘッダがあれば上書きしない（LLM修復を尊重）
    - .fixed_title_from_stem/ にマーカーを書き、再実行は未処理のみ
    """
    cmap = _load_chunk_map_dict()
    fixed = 0
    scanned = 0

    for cf in sorted(Path(CHUNK_DIR).glob("*.txt")):
        scanned += 1
        marker = FIXED_DIR / (cf.stem + ".fixed")
        if marker.exists():
            continue

        try:
            lines = open(cf, encoding="utf-8").read().splitlines()
        except Exception:
            continue
        if not lines:
            marker.write_text("skip-empty");
            continue

        head = (lines[0] or "").strip()

        # ★ 既に三項ヘッダが正しい形式なら何もしない（LLM修復を尊重）
        if HEADER_OK.match(head):
            marker.write_text("ok-skip")
            continue

        # src_txt を解決（マップ優先→推測）
        src_txt = cmap.get(str(cf)) or cmap.get(cf.name)
        if not src_txt:
            name_part = cf.stem.split("__")[0]
            guess = Path(TEXT_DIR) / f"{name_part}.txt"
            src_txt = str(guess) if guess.exists() else None

        # タイトルを決定
        if not src_txt or not Path(src_txt).exists():
            title = unicodedata.normalize("NFKC", cf.stem.split("__")[0]).strip()
        else:
            title = _title_from_src_txt(src_txt)

        new_hdr = _new_header(title, style=style)

        if dry_run:
            print(f"[DRY] {cf.name}: '{head}' -> '{new_hdr}'")
            marker.write_text("dry-run")
        else:
            # ここからは「ヘッダが無い/壊れている」場合のみ補完
            if head.startswith("【"):
                lines[0] = new_hdr
            else:
                lines.insert(0, new_hdr)
            with open(cf, "w", encoding="utf-8") as f:
                f.write("\n".join(lines))
            marker.write_text("fixed")
            fixed += 1

        if not dry_run and fixed >= batch:
            break

    print(f"fix_chunk_headers_from_filename: scanned={scanned}, fixed={fixed}, style={style}, dry_run={dry_run}")


In [ ]:
fix_chunk_headers_from_filename(CHUNK_DIR, TEXT_DIR, style="triple", dry_run=False, batch=5000)

fix_chunk_headers_from_filename: scanned=5000, fixed=5000, style=triple, dry_run=False


In [27]:
import regex as rex            # ← 標準 re は残し、\p を使う所は rex を使う
from collections import Counter

ABSTRACT_PAT = rex.compile(r"(要点の整理|説明せよ|述べよ|概要|考察|まとめ|要約せよ|解説せよ)")

In [28]:
import os, json, re
from pathlib import Path

ENT_DIR = Path(CHUNK_DIR) / ".ents"
ENT_DIR.mkdir(exist_ok=True)

# 軽い正規表現の種（ドメイン向け）
RE_STD  = re.compile(r"(?:IEEE\s*802\.\d+[a-z]?|802\.\d+[a-z]?|ITU-?T|IEC|OIF|RS-?FEC|PAM4|NRZ|OTN|SDH|GPON|XGSPON)", re.I)
RE_NUM  = re.compile(r"\b\d+(?:\.\d+)?\s*(?:Gbit/s|Gbps|Tbps|Mb/s|nm|dBm|Clause\s*\d+|%)\b", re.I)
RE_ORG  = re.compile(r"\b(NTT|IEEE|IEC|OIF|ISO|IETF|ETSI|3GPP|JEITA|JSA)\b")
RE_TECH = re.compile(r"(?:イーサネット|Ethernet|光モジュール|ROADM|EDFA|APD|WDM|DWDM|CWDM|MAC層|物理層)", re.I)

def _regex_entities(text:str):
    text = j_normalize(text)
    std  = sorted(set(RE_STD.findall(text)))
    num  = sorted(set(RE_NUM.findall(text)))
    org  = sorted(set(RE_ORG.findall(text)))
    tech = sorted(set(RE_TECH.findall(text)))
    return {"standards": std, "nums": num, "orgs": org, "tech": tech}

def _topics_via_llm(text:str, hint_terms:list[str], limit=6):
    # LLMには“ラベル列挙のみ”をさせる（暴走防止）
    sys = "あなたは要約タグ付け器です。本文の主題トピックを名詞句で0〜6個、JSON配列のみで返してください。説明文は禁止。"
    user = (
        "本文抜粋（先頭〜）：\n---\n" + j_normalize(text)[:1200] + "\n---\n"
        + ("ヒント語: " + "、".join(hint_terms[:20]) if hint_terms else "")
    )
    out = gen([{"role":"system","content":sys},{"role":"user","content":user}], max_new_tokens=120, do_sample=False)
    m = re.search(r"\[.*\]\s*$", out, re.S)
    try:
        arr = json.loads(m.group(0)) if m else []
        # 名詞句だけを簡単にフィルタ
        arr = [t.strip() for t in arr if isinstance(t, str) and 2 <= len(t.strip()) <= 40]
        return arr[:limit]
    except Exception:
        return []

def extract_and_store_entities(chunk_id:str, chunk_text:str):
    ents = _regex_entities(chunk_text)
    hint = sum((ents.get(k,[]) for k in ("standards","orgs","tech")), [])
    topics = _topics_via_llm(chunk_text, hint_terms=hint, limit=6)
    payload = {"entities": ents, "topics": topics}

    # ★ chunk_id が無い/暫定なら永続化しない
    if not chunk_id or chunk_id == "transient":
        return payload

    outp = ENT_DIR / f"{chunk_id}.json"
    with open(outp, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)
    return payload


def load_entities(chunk_id:str):
    fp = ENT_DIR / f"{chunk_id}.json"
    if fp.exists():
        try:
            return json.load(open(fp, encoding="utf-8"))
        except Exception:
            return None
    return None


In [ ]:
#one = next(iter(sorted(Path(CHUNK_DIR).glob("*.txt"))))
#process_chunk_resume_safe(str(one))
#!tail -n 5 "$Q_FILE"


tail: cannot open '/content/mydrive/MyDrive/phi4_proj_23/sft_build/out/questions.jsonl' for reading: No such file or directory


In [28]:
chunks = sorted(Path(CHUNK_DIR).glob("*.txt"))
if chunks:
    process_chunk_resume_safe(str(chunks[0]))
    print("tail Q_FILE:")
    !tail -n 5 "$Q_FILE"
else:
    print("[info] No chunks yet. Skipped sample Q/A.")

NameError: name 'process_chunk_resume_safe' is not defined

In [29]:
# 既存: import os, json, re  ... の直後に追記
import regex as rex  # ← これを追加（標準reはそのまま残す）
from collections import Counter


In [33]:
def _parse_llm_json(out: str, prefer="object"):
    """
    LLMの出力から最後のJSONだけを安全に抽出してパースする。
    - ```json ... ``` や前後の説明文を除去
    - {...} 優先、無ければ [...]
    - 末尾の余計なカンマも軽く修正
    失敗時は ValueError を投げる
    """
    s = (out or "").strip()
    # コードフェンス除去
    s = re.sub(r"^```(?:json)?\s*|\s*```$", "", s, flags=re.S)
    # 全体から最後の {} を抜く
    if prefer in ("object","auto"):
        l = s.find("{"); r = s.rfind("}")
        if l != -1 and r != -1 and r > l:
            cand = s[l:r+1]
            try:
                return json.loads(cand)
            except json.JSONDecodeError:
                # 末尾カンマを緩和
                cand2 = re.sub(r",\s*([}\]])", r"\1", cand)
                return json.loads(cand2)
    # 配列優先モード or オブジェクトが見つからない場合は [] を試す
    l = s.find("["); r = s.rfind("]")
    if l != -1 and r != -1 and r > l:
        cand = s[l:r+1]
        try:
            return json.loads(cand)
        except json.JSONDecodeError:
            cand2 = re.sub(r",\s*([}\]])", r"\1", cand)
            return json.loads(cand2)
    raise ValueError("LLM JSON parse failed")


In [34]:
# 先頭の imports 付近に
import math

SPLIT_SENT = rex.compile(r"[。．\.!?？！]\s*|\n+")

def _sentences(text: str):
    text = j_normalize(text)
    return [t.strip()[:800] for t in SPLIT_SENT.split(text) if t and t.strip()]

def _keywords_from_question(q: str):
    qn = j_normalize(q)
    # 「…」内は強キーワード、数値・単位、英数字語を抽出
    quoted = rex.findall(r"「([^」]{2,})」", qn)
    nums   = rex.findall(r"\d+(?:\.\d+)?\s*(?:Gbit/s|Gbps|Tbps|Mb/s|nm|dBm|Clause\s*\d+|%|台|件)", qn, flags=rex.I)
    words  = rex.findall(r"[A-Za-z0-9\.\-]{3,}", qn)
    # 日本語の重要語（2文字以上の漢字/カタカナ）
    jwords = rex.findall(r"[一-龥]{2,}|[ァ-ヶｦ-ﾟー]{2,}", qn)
    return list(dict.fromkeys(quoted + nums + words + jwords))  # 重複排除

def _score_sentence(sent: str, keys: list[str]) -> float:
    s = j_normalize(sent)
    if not s: return 0.0
    score = 0.0
    for k in keys:
        if not k: continue
        if k in s:
            # 「…」内や数値っぽいキーは重み高く
            w = 3.0 if rex.search(r"[\"”」]?\b?\d", k) or len(k) >= 6 else 2.0
            score += w
    # 数値＋単位が含まれる文は少しブースト
    if rex.search(r"\d+(?:\.\d+)?\s*(?:Gbit/s|Gbps|Tbps|Mb/s|nm|dBm|Clause\s*\d+)", s, flags=rex.I):
        score += 1.5
    # IEEE/OTN/Ethernet等ドメイン語の軽ブースト
    if rex.search(r"(IEEE|802\.3|OTN|Ethernet|イーサネット|ITU-?T|OIF|IEC)", s, flags=rex.I):
        score += 1.0
    return score

def retrieve_support_spans(chunk_text: str, question: str, topk: int = 6) -> list[str]:
    sents = _sentences(chunk_text)
    keys = _keywords_from_question(question)
    scored = sorted(((s, _score_sentence(s, keys)) for s in sents), key=lambda x: x[1], reverse=True)
    hits = [s for s, sc in scored if sc > 0][:topk]
    # 余りに短い破片は除外
    hits = [h for h in hits if len(h) >= 10]
    return hits


In [35]:
# ★ 先頭の imports 直後（regex as rex の前でも後でもOK）に追加
try:
    JP_STOP
except NameError:
    JP_STOP = set()

JP_STOP |= {
    "は","が","を","に","へ","と","で","や","も","から","まで","より","など","の",
    "第","章","節","項","図","表","例","同","本","本文","本稿","前述","後述",
    "これ","それ","どれ","もの","こと"
}


# ドメイン強化（Ethernet/OTN/単位などを網羅）
DOMAIN_PAT = rex.compile(
    r"(?:IEEE\s*802\.3[a-z]?(?:\s*Clause\s*\d+)?|802\.3[a-z]?|Ethernet|イーサネット|OTN|OTU[1-4]|"
    r"RS-?FEC|FEC|LDPC|Reed-?Solomon|PAM4|NRZ|CDR|DSP|PCS|PMA|MAC|MDI|XGMII|"
    r"BER|ビット誤り率|PRBS\d+|QSFP(?:28|56)?|SFP\+|ROADM|EDFA|APD|PIN|WDM|DWDM|CWDM|"
    r"GPON|XG-?S?PON|PON|OFDM|"
    r"\d+(?:\.\d+)?\s*(?:Gbps?|Gbit/s|Tbps?|Mb/s|nm|dBm|ps/nm/km|UI))",
    rex.I
)

def _good_term(t: str) -> bool:
    if not t: return False
    t = t.strip()
    if t in JP_STOP: return False

    # 1文字は原則NG（漢字1文字も落とす）
    if len(t) == 1: return False

    # ASCIIは英数3文字以上
    if rex.search(r"[A-Za-z0-9]", t):
        if len(rex.sub(r"[^A-Za-z0-9]", "", t)) < 3:
            return False

    # 強いドメイン語は即OK
    if DOMAIN_PAT.search(t): return True

    # 日本語：漢字2字以上 or カタカナ2字以上のみ
    if rex.search(r"[一-龥]", t) and len(t) >= 2: return True
    if rex.fullmatch(r"[ァ-ヶｦ-ﾟー]{2,}", t): return True

    return False

def _extract_terms(chunk_text, topk=80):
    chunk_text = j_normalize(chunk_text)
    strong = DOMAIN_PAT.findall(chunk_text)
    strong = [s.strip() for s in strong if _good_term(s)]

    toks = rex.findall(
        r"[A-Za-z0-9_.:/-]{3,}|\p{Script=Han}+|\p{Script=Katakana}+|\p{Script=Hiragana}+",
        chunk_text, flags=rex.UNICODE
    )
    cnt = Counter(toks)
    base = [w for w,_ in cnt.most_common(800) if _good_term(w)]

    seen = set()
    ordered = [t for t in (strong + base) if not (t in seen or seen.add(t))]
    return ordered[:topk]


In [58]:
SPAN_SYS = """
あなたはエビデンス採掘係です。与えられた本文（context）から、
質問の根拠に使える「逐語引用スパン」だけを抽出します。
- 出力は JSON のみ。
- スパンは本文の文字列と完全一致。改変しない。
- 各スパンに type を付与: "def"（定義/位置づけ）, "num"（数値＋単位）, "cmp"（対比が示唆）, "causal"（因果）, "proc"（手順）。
- 根拠が不十分なら空配列 [] を返す。
- 前後に説明文を付けない。コードフェンス禁止。JSONのみ。
""".strip()

def mine_spans_with_llm(chunk_text: str, max_new_tokens=420):
    txt = j_normalize(chunk_text)
    user = (
        "context:\n---\n" + txt[:6000] + "\n---\n"
        '返答は厳格JSONのみ。{"spans":[{"id":"S1","type":"def|num|cmp|causal|proc","quote":"…引用…"}]}'
    )
    out = gen([{"role":"system","content":SPAN_SYS},{"role":"user","content":user}],
              max_new_tokens=max_new_tokens, do_sample=False)

    try:
        data = _parse_llm_json(out, prefer="object")
    except Exception:
        print("[mine_spans_with_llm] JSON parse failed. Raw head:", (out or "")[:200])
        data = {"spans":[]}

    data = _as_obj(data, "spans")
    spans, seen = [], set()
    for it in _get_list(data, "spans"):
        if not isinstance(it, dict):
            continue
        q = j_normalize((it.get("quote") or "").strip())
        t = (it.get("type")  or "").strip().lower()
        sid = (it.get("id")  or "").strip() or f"S{len(spans)+1}"
        if len(q) >= 10 and q in txt and t in {"def","num","cmp","causal","proc"} and sid not in seen:
            spans.append({"id": sid, "type": t, "quote": q})
            seen.add(sid)
    return spans[:10]

In [59]:
QGEN_FROM_SPAN_SYS = """
あなたは質問起草係です。与えられた spans（逐語引用）だけを根拠に、学習者の専門知識を鍛える良問を作ります。
- spans 以外の本文・一般知識を根拠にしない。根拠不足なら作らない。
- 最大 5 問。0 問可。
- 各質問は参照した span の id を "use" に配列で列挙する。
- 出力は JSON のみ：{"questions":[{"q":"...","use":["S1","S2"]}]}。
- 前後に説明文を付けない。コードフェンス禁止。JSONのみ。
""".strip()

def draft_questions_from_spans(spans: list, title="この文書", max_new_tokens=360):
    if not spans:
        return []
    user = (
        f"タイトル参考: {title}\n"
        "利用できる spans:\n" + json.dumps({"spans":spans}, ensure_ascii=False) + "\n"
        '返答は厳格JSONのみ。{"questions":[{"q":"...","use":["S1","S2"]}]}'
    )
    out = gen([{"role":"system","content":QGEN_FROM_SPAN_SYS},{"role":"user","content":user}],
              max_new_tokens=max_new_tokens, do_sample=False)

    try:
        data = _parse_llm_json(out, prefer="object")
    except Exception:
        print("[draft_questions_from_spans] JSON parse failed. Raw head:", (out or "")[:200])
        data = {"questions":[]}

    data = _as_obj(data, "questions")
    qs = []
    for it in _get_list(data, "questions"):
        if not isinstance(it, dict):
            continue
        q   = j_normalize((it.get("q") or "").strip())
        use = [u for u in (it.get("use") or []) if isinstance(u, str)]
        if len(q) >= 12 and use:
            qs.append({"q": q, "use": use[:3]})
    return qs[:5]


In [56]:
def _as_obj(data, top_key: str):
    """LLM出力が list の時も dict にラップして返す。Noneやスカも空扱い。"""
    if isinstance(data, dict):
        return data
    if isinstance(data, list):
        return {top_key: data}
    return {top_key: []}

def _get_list(d: dict, key: str):
    """dict.get(...) が list じゃない場合も空配列にフォールバック。"""
    v = (d or {}).get(key, [])
    return v if isinstance(v, list) else []


In [60]:
def factcheck_against_spans(chunk_text: str, qs_with_use: list, spans: list):
    txt = j_normalize(chunk_text)

    # 機械チェック
    span_by_id = {s["id"]: s for s in spans}
    kept = []
    for it in qs_with_use:
        use_ids = [u for u in it["use"] if u in span_by_id]
        quotes  = [span_by_id[u]["quote"] for u in use_ids]

        if len(quotes) < 2 or not all(q in txt for q in quotes):
            continue

        needs_num = bool(re.search(r"(数値|閾値|Gbps|Gbit/s|nm|dBm|Clause\s*\d+)", it["q"], re.I))
        if needs_num and not any(re.search(r"\d+(?:\.\d+)?\s*(Gbit/s|Gbps|Tbps|Mb/s|nm|dBm|Clause\s*\d+|%)", q, re.I) for q in quotes):
            continue

        needs_cmp = "比較" in it["q"]
        if needs_cmp and not any(span_by_id[u]["type"] == "cmp" for u in use_ids):
            continue

        kept.append(it)

    if not kept:
        return []

    # LLMチェック
    SYS = (
        "あなたはファクトチェック係です。指定の引用（quotes）だけを根拠に、"
        "その質問が本文の範囲で答えられるか判定します。"
        "出力はJSONのみ（ok: true/false, reason: string）。前後に文章を付けない。コードフェンス禁止。"
    )
    vetted = []
    for it in kept:
        payload = {
            "question": it["q"],
            "quotes": [span_by_id[u]["quote"] for u in it["use"]]
        }
        out = gen(
            [{"role":"system","content":SYS},
             {"role":"user","content":json.dumps(payload, ensure_ascii=False)}],
            max_new_tokens=140, do_sample=False
        )

        ok = False
        try:
            d = _parse_llm_json(out, prefer="object")
            d = _as_obj(d, "ok")
            ok = str((d.get("ok") or "")).lower() in {"true","yes","y","1"}
        except Exception:
            print("[factcheck_against_spans] JSON parse failed. Raw head:", (out or "")[:200])
            ok = False

        if ok:
            vetted.append(it)

    return vetted

In [39]:
FORMAT_SYS = "あなたは整形係です。質問文を最小限で読みやすく整えます。意味の追加や一般化はしない。出力は質問文のみ。"

def finalize_questions(vetted, prefix=None, bank_terms=None, max_out=5):
    out=[]
    for it in vetted:
        q0 = it["q"]
        # 用語ガード（docのbank語が使える場合）
        if bank_terms and not any(t in q0 for t in bank_terms[:40]):
            continue
        # LLMで最小修正
        q1 = gen([{"role":"system","content":FORMAT_SYS},{"role":"user","content":q0}],
                 max_new_tokens=100, do_sample=False).strip()
        if prefix and not q1.startswith("これは"):
            q1 = f"{prefix}{q1}"
        out.append(q1)
        if len(out) >= max_out: break
    return out


In [40]:
ABSTRACT_PAT = rex.compile(r"(要点の整理|説明せよ|述べよ|概要|考察|まとめ|要約せよ|解説せよ)")

def _validate_questions(chunk_text, title, questions, require=5):
    chunk_text = j_normalize(chunk_text)
    terms = _extract_terms(chunk_text, topk=80)
    def has_term(q): return any(t in q for t in terms[:40])
    def has_anchorish(q):
        keys = rex.findall(r"「([^」]{2,})」", q)
        return any(k in chunk_text for k in keys) if keys else True

    seen, cleaned = set(), []
    for q in questions:
        short_quoted = any(len(m) <= 2 for m in rex.findall(r"「([^」]+)」", q))
        k = rex.sub(r"\W+", "", q.lower())
        if len(q) >= 12 and not ABSTRACT_PAT.search(q) and not short_quoted and has_term(q) and has_anchorish(q) and k not in seen:
            cleaned.append(q); seen.add(k)

    return cleaned[:require]



In [ ]:
#QUESTION_POLICY = """
#あなたは『通信ネットワーク／光伝送／Ethernet/OTN/IEEE 802.3/ITU-T』領域の専門出題者です。
#目的：本文の「専門知識・規格・数値・因果」を鍛える具体質問のみを作る。
#厳守事項：
#- 本文に根拠がある内容のみ。本文に無い事実・数値・固有名詞は出さない。
#- 各設問には本文に現れる**専門語**（規格名・方式名・項番・数値・単位）を**最低1つ**必ず含める。
#- 次のホワイトリストの語から**少なくとも1つ**を必ず含める：{terms_hint}
#- 抽象表現（「要点の整理」「説明せよ」「述べよ」等）禁止。
#- 5観点を優先：定義／比較／数値（単位つき）／因果／適用・手順。
#- 冒頭に必ず「これは『{title}』に関する質問です。」を入れる。
#出力形式：厳格JSONのみ。例：{{"questions":["...","...","...","...","..."]}}
#""".strip()


In [41]:
SOFT_QUESTION_POLICY = """
あなたは『通信ネットワーク／光伝送／Ethernet/OTN/IEEE 802.3/ITU-T』領域の出題アシスタントです。
目的：渡された本文抜粋（context）に**直接の根拠（逐語引用）**があるときだけ、専門語を含む具体設問を**0〜5問**作る。

厳守事項：
- 無理に作らない。根拠がなければ作らない（0問可）。
- 各設問は本文に現れる**専門語**（規格名・方式名・項番・数値・単位など）を**1つ以上**含む。
- 各設問には**anchors（逐語引用, 10〜80文字, 2本以上）**を必ず付ける。引用は**contextと完全一致**。
- 設問typeは次のいずれか： "def"（定義）, "cmp"（比較）, "num"（数値）, "causal"（因果）, "proc"（手順）。
- typeごとの追加条件：
  - "def": anchorsの少なくとも1本が**「<用語> とは…/… defined as …」型**の定義文脈であること。
  - "cmp": anchorsの**片方に用語A、もう片方に用語B**を含める（同一anchor内で両語でも可）。
  - "num": anchorsのどちらかに**数値+単位**が含まれること（例: 100 Gbit/s, Clause 49, dBm など）。
- 出力は厳格JSONのみ：
  { "questions": [ { "q": "...", "type":"cmp|def|num|causal|proc", "anchors": ["...","..."] } ] }
""".strip()


In [42]:
def j_normalize(s: str) -> str:
    if not s: return ""
    s = unicodedata.normalize("NFKC", s)
    s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")
    # カタカナの不自然な空白を除去
    s = re.sub(r"([ァ-ヶｦ-ﾟ])\s+(?=[ァ-ヶｦ-ﾟ])", "", s)
    # ★ 追加：語中ソフト改行の吸収（例: IEEE \n802.3 → IEEE 802.3）
    s = re.sub(r'(?<=[A-Za-z0-9\.\-])\s*\n\s*(?=[A-Za-z0-9\.\-])', ' ', s)
    # 異常な不可視空白の圧縮
    s = re.sub(r"[\u2000-\u200F\u2028\u2029]", " ", s)
    return s


In [43]:
SPLIT_SENT = rex.compile(r"[。．\.!?？！]\s*|\n+")

DEF_PAT = rex.compile(r"(とは|とは何か|定義|とは、|とは – |is\s+(?:a|an|the)|defined\s+as)", rex.I)
NUM_PAT = rex.compile(r"\b\d+(?:\.\d+)?\s*(?:Gbps?|Gbit/s|Tbps?|Mb/s|nm|dBm|ps/nm/km|UI|Clause\s*\d+)\b", rex.I)

def _sentences(text: str):
    text = j_normalize(text)
    # 粗く文に分割
    raw = [t.strip() for t in SPLIT_SENT.split(text) if t and t.strip()]
    # 長すぎ断片を短縮
    return [t[:1200] for t in raw]

def _has_def_evidence(term: str, sent: str) -> bool:
    return (term in sent) and bool(DEF_PAT.search(sent))

def _has_pair_evidence(t1: str, t2: str, sent: str) -> bool:
    return (t1 in sent) and (t2 in sent)

def _numeric_items(sent: str):
    return NUM_PAT.findall(sent)

def _best_terms_for_chunk(chunk_text: str, max_terms=40):
    # 既存の _extract_terms を活用
    terms = _extract_terms(chunk_text, topk=80)
    # 助詞や短語は _good_term で既に排除済みの想定
    return terms[:max_terms]


In [44]:
# ===== Resume-safe Q/A pipeline helpers =====
import os, json, re
from pathlib import Path

# 既存ログを読む（壊れ行はスキップ）
def _read_jsonl(path):
    items = []
    if not os.path.exists(path): return items
    with open(path, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line: continue
            try:
                items.append(json.loads(line))
            except Exception:
                continue
    return items

def _qnum(qid):
    m = re.search(r'#Q(\d+)', qid or '')
    return int(m.group(1)) if m else 10**9

def _existing_questions(chunk_id):
    rows = _read_jsonl(Q_FILE)
    qs = [(r.get("q_id"), r.get("question")) for r in rows
          if r.get("chunk_id")==chunk_id and r.get("question")]
    qs.sort(key=lambda t: _qnum(t[0]))
    return qs  # list[(q_id, question)]

def _answered_qids():
    s = set()
    for fp in (ANS_FILE, SFT_FILE):
        for r in _read_jsonl(fp):
            qid = r.get("q_id")
            if qid: s.add(qid)
    return s

def _norm(s):  # 質問の重複判定用
    return re.sub(r'\W+', '', (s or '').lower())


# チャンク1個を再開安全に処理：既存Qは再利用し、不足分だけ補充→未回答だけ回答生成
QA_DONE_DIR = f"{CHUNK_DIR}/.qa_done"
os.makedirs(QA_DONE_DIR, exist_ok=True)

import hashlib, unicodedata
def _safe_qa_marker_path(chunk_path: str) -> Path:
    stem = Path(chunk_path).stem
    s = unicodedata.normalize("NFKC", stem)
    s = re.sub(r'[\/\\:\*\?"<>\|\n\r\t]', "_", s).strip(" .")
    s = s[:80] if len(s) > 80 else (s or "untitled")
    h = hashlib.md5(stem.encode("utf-8")).hexdigest()[:8]
    return Path(QA_DONE_DIR) / f"{s}.{h}.done"

def process_chunk_resume_safe(chunk_path, require_questions=5, max_self_checks=2):
    chunk_path = str(chunk_path)
    chunk_id   = Path(chunk_path).stem
    marker     = _safe_qa_marker_path(chunk_path)
    if marker.exists():
        return  # すでにQ/A完了済み

    chunk_text = open(chunk_path, encoding="utf-8").read()

    # タイトル抽出
    first_line  = (chunk_text.splitlines()[0].strip() if chunk_text else "")
    chunk_title = re.sub(r"[【】]", "", first_line).split("｜")[0] if first_line.startswith("【") else "この文書"

    # 1) 既存Q
    existing = _existing_questions(chunk_id)
    q_map    = {qid: q for qid, q in existing}
    qnorms   = {_norm(q) for q in q_map.values()}

    # 2) 不足分だけ補充（タイトル渡す）
    if len(existing) < require_questions:
        effective_title, doc_memo = resolve_effective_title_and_memo(chunk_path, chunk_text)
        bank = []
        for k in ("standards","orgs","tech"):
          bank += (doc_memo.get("entities",{}) or {}).get(k, []) or []
        bank = list(dict.fromkeys(bank))

        # b) エビデンス採掘（LLM）
        spans = mine_spans_with_llm(chunk_text)

        # c) 質問起草（LLM; span縛り）
        drafted = draft_questions_from_spans(spans, title=effective_title)

       # d) ファクトチェック（LLM＋機械）
        vetted = factcheck_against_spans(chunk_text, drafted, spans)

      # e) 整形（LLM; 最小修正）
        prefix = f"これは【{effective_title}】に関する質問です。"
        props  = finalize_questions(vetted, prefix=prefix, bank_terms=bank, max_out=require_questions)

        terms = _extract_terms(chunk_text, topk=80)
        strong_terms = set(terms[:30])

        next_idx = max([_qnum(qid) for qid,_ in existing] + [0]) + 1
        with open(Q_FILE, "a", encoding="utf-8") as f:
            for q in props:
               if not any(t and (t in q) for t in strong_terms):
                   continue
               if _norm(q) in qnorms:
                   continue
               if next_idx > require_questions:
                   break
               qid = f"{chunk_id}#Q{next_idx}"
               f.write(json.dumps({"chunk_id":chunk_id, "q_id": qid, "question": q}, ensure_ascii=False) + "\n")
               q_map[qid] = q
               qnorms.add(_norm(q))
               next_idx += 1

In [45]:
# === Ensure Q/A functions exist (re-define if missing) ===
import re, json

def answer_with_context(chunk_text, question, title="この文書", max_chars=900):
    chunk_text = j_normalize(chunk_text)
    question   = j_normalize(question)

    # ★ サポート文抽出
    ctx_sents = retrieve_support_spans(chunk_text, question, topk=6)
    ctx = "\n".join(ctx_sents)

    # 根拠が全く拾えなかったら回答不能を返す（テンプレ空返しを防ぐ）
    if not ctx_sents:
        return "結論: 回答不能（本文該当箇所が見当たりません）\n根拠: - 「該当文なし」\n手順/考え方: 1) 質問語に一致する文なし\n制約: 本文の範囲のみ"

    sys = (
        "あなたは与えられた本文抜粋（context）の**範囲内だけ**で答えるアシスタントです。"
        "必ず**5文字以上の逐語引用**を**2つ以上**含めてください（全角/半角混在可）。"
        "contextに無い主張・定義・数値は出さない。無理なら『回答不能（本文根拠なし）』とする。"
        f"全体は{max_chars}文字以内。最終出力は <final> ... </final> の間だけ。"
    )
    user = (
        f"本文タイトル: 『{title}』\n"
        "context（回答に使ってよい根拠文のみ）:\n<<<\n" + ctx + "\n>>>\n"
        f"質問: {question}\n\n"
        "出力テンプレ:\n"
        "<final>\n"
        "結論: <一言で結論/不可なら『回答不能（本文根拠なし）』>\n"
        "根拠:\n"
        "- 「<contextからの逐語引用1（5文字以上）>」の要約\n"
        "- 「<contextからの逐語引用2（5文字以上）>」の要約\n"
        "手順/考え方: 1) ... 2) ... 3) ...\n"
        "制約: <本文の前提/制約。無ければ「本文に記載なし」>\n"
        "</final>"
    )
    out = gen([{"role":"system","content":sys},{"role":"user","content":user}],
              max_new_tokens=720, do_sample=False)
    m = re.search(r"<final>\s*(.+?)\s*</final>", out, flags=re.S)
    body = (m.group(1).strip() if m else out.strip())

    # 文字数制限
    return (body[:max_chars] + "…") if len(body) > max_chars else body

def self_check(chunk_text, question, answer, title="この文書"):
    chunk_text = j_normalize(chunk_text)
    question   = j_normalize(question)
    ctx_sents  = retrieve_support_spans(chunk_text, question, topk=8)
    ctx        = "\n".join(ctx_sents)

    #quotes = re.findall(r'「([^」]{5,200})」', answer)  # ★ 5文字以上に強化
    quotes = re.findall(r'[「“"]([^」”"]{5,200})[」”"]', answer)
    has_two_quotes = len([q for q in quotes if q in ctx]) >= 2
    has_conclusion = bool(re.search(r'結論\s*:', answer))
    nonempty = bool(answer.strip())

    # 数値/閾値系なら、少なくとも1つの数値＋単位が引用に含まれること
    needs_numeric = ("数値" in question) or ("閾値" in question)
    num_ok = (not needs_numeric) or any(re.search(r"\d+(?:\.\d+)?\s*(?:Gbit/s|Gbps|Tbps|Mb/s|nm|dBm|Clause\s*\d+|%)", q, flags=re.I) for q in quotes)

    if nonempty and has_conclusion and has_two_quotes and num_ok:
        return "ACCEPT", answer.strip(), "LOCAL_ACCEPT"

    # LLMでの再整形（contextのみ参照、引用2本必須）
    sys = (
        "与えられたcontextと回答だけを参照して、本文根拠に沿うよう最小修正。"
        "必ず**5文字以上の逐語引用**を**2つ以上**含める。"
        "contextに無い内容は削除。最終出力は<final>のみ。"
    )
    user = (
        f"本文タイトル: 『{title}』\ncontext:\n<<<\n{ctx}\n>>>\n"
        f"質問: {question}\n"
        f"回答:\n<<<\n{answer}\n>>>\n"
        "テンプレ:\n<final>\n結論: ...\n根拠:\n- 「...」...\n- 「...」...\n手順/考え方: 1) ... 2) ...\n制約: ...\n</final>"
    )
    out = gen([{"role":"system","content":sys},{"role":"user","content":user}], max_new_tokens=720, do_sample=False)
    m = re.search(r"<final>\s*(.+?)\s*</final>", out, flags=re.S)
    revised = m.group(1).strip() if m else answer.strip()

    # 引用確認をもう一度
    quotes2 = re.findall(r'「([^」]{5,200})」', revised)
    has_two_quotes2 = len([q for q in quotes2 if q in ctx]) >= 2
    if revised and ("結論:" in revised) and has_two_quotes2 and (num_ok or not needs_numeric):
        return "REVISE", revised, out

    # 最後の保険
    fallback = "結論: 回答不能（本文根拠なし）\n根拠: - 「該当文なし」\n手順/考え方: 1) context精査 2) 不一致のため中止\n制約: 本文の範囲のみ"
    return "REVISE", fallback, out



In [46]:
# 1) どこに書く想定か
print("Q_FILE:", Q_FILE)
print("ANS_FILE:", ANS_FILE)
print("SFT_FILE:", SFT_FILE)

# 2) 本当に書かれていないか（行数）
import os
for p in [Q_FILE, ANS_FILE, SFT_FILE]:
    print(p, "exists?", os.path.exists(p), "lines:", sum(1 for _ in open(p, encoding="utf-8")) if os.path.exists(p) else 0)

# 3) 全チャンクのうち、Q/A 完了マーカーが既に付いていてスキップされてないか
from pathlib import Path
qa_done = list(Path(f"{CHUNK_DIR}/.qa_done").glob("*.done"))
print(".qa_done count:", len(qa_done), "例:", [p.name[:50] for p in qa_done[:5]])


Q_FILE: /content/mydrive/MyDrive/phi4_proj_23/sft_build/out/questions.jsonl
ANS_FILE: /content/mydrive/MyDrive/phi4_proj_23/sft_build/out/answers.jsonl
SFT_FILE: /content/mydrive/MyDrive/phi4_proj_23/sft_build/out/dataset.jsonl
/content/mydrive/MyDrive/phi4_proj_23/sft_build/out/questions.jsonl exists? False lines: 0
/content/mydrive/MyDrive/phi4_proj_23/sft_build/out/answers.jsonl exists? False lines: 0
/content/mydrive/MyDrive/phi4_proj_23/sft_build/out/dataset.jsonl exists? False lines: 0
.qa_done count: 3 例: ['03_05_fiÁ‘W.qxd__00000.0e0ac1a9.done', '03_05_fiÁ‘W.qxd__00001.93fe1793.done', '03_05_fiÁ‘W.qxd__00002.8542b125.done']


抽象質問の除去＆差し替え（一度やるだけの整備）

既にQ_FILEに入った「本文要点の整理…」を消し、タイトル付きの具体質問で埋め戻します。

In [47]:
import regex as rex
from collections import Counter


In [48]:
DOC_ENT_DIR = Path(TEXT_DIR) / ".docents"
DOC_ENT_DIR.mkdir(exist_ok=True)

def _doc_key_from_src(src_txt_path: str) -> str:
    s = str(Path(src_txt_path).resolve())
    return hashlib.md5(s.encode("utf-8")).hexdigest()[:16]

def extract_doc_entities(src_txt_path: str, limit_chars=120000):
    text = open(src_txt_path, encoding="utf-8", errors="ignore").read()
    text = j_normalize(text)[:limit_chars]
    base = _regex_entities(text)  # ←既存の正規表現抽出を利用
    topics = _topics_via_llm(
        text[:4000],
        hint_terms=sum((base.get(k,[]) for k in ("standards","orgs","tech")), []),
        limit=8
    )
    first = (text.splitlines()[0].strip() if text else "")
    hdr_ok = first.startswith("【") and "｜" in first
    doc_title_raw = re.sub(r"[【】]", "", first).split("｜")[0] if hdr_ok else ""
    return {"entities": base, "topics": topics, "doc_title_raw": doc_title_raw}

def load_or_build_doc_entities(src_txt_path: str):
    key = _doc_key_from_src(src_txt_path)
    fp = DOC_ENT_DIR / f"{key}.json"
    if fp.exists():
        try:
            return json.load(open(fp, encoding="utf-8"))
        except Exception:
            pass
    data = extract_doc_entities(src_txt_path)
    with open(fp, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    return data


In [49]:
ABSTRACT = re.compile(r"(本文要点の整理|説明せよ|述べよ|概要|考察|まとめ|要約せよ)")
def replace_abstract_questions(chunk_text, chunk_id):
    # 1) 既存Qを読み、抽象質問だけ拾う
    first = (chunk_text.splitlines()[0].strip() if chunk_text else "")
    title = re.sub(r"[【】]", "", first).split("｜")[0] if first.startswith("【") else "この文書"

    rows = _read_jsonl(Q_FILE)
    keep, bad = [], []
    for r in rows:
        if r.get("chunk_id")==chunk_id and ABSTRACT.search(str(r.get("question",""))):
            bad.append(r)
        else:
            keep.append(r)
    if not bad:
        return False

    # 2) 【ここを旧版から置き換え】span縛りで新しい具体質問を作成
    #    - 有効タイトル（壊れていれば擬似タイトル）＋ docメモリ
    try:
        # chunkファイルの実体パスを推定（CHUNK_DIR/{chunk_id}.txt）
        chunk_fp_guess = str(Path(CHUNK_DIR) / f"{chunk_id}.txt")
        effective_title, doc_memo = resolve_effective_title_and_memo(chunk_fp_guess, chunk_text)
    except Exception:
        effective_title, doc_memo = (title or "この文書"), {"entities":{}, "topics":[]}

    # docレベルの語彙バンク（ガード用）
    bank = []
    for k in ("standards","orgs","tech"):
        bank += (doc_memo.get("entities",{}) or {}).get(k, []) or []
    bank = list(dict.fromkeys(bank))

    # エビデンス採掘 → span縛り起草 → ファクトチェック → 整形
    spans   = mine_spans_with_llm(chunk_text)
    drafted = draft_questions_from_spans(spans, title=effective_title)
    vetted  = factcheck_against_spans(chunk_text, drafted, spans)
    prefix  = f"これは【{effective_title}】に関する質問です。"
    props   = finalize_questions(vetted, prefix=prefix, bank_terms=bank, max_out=len(bad))

    # 3) 書き戻し（バックアップを取ってから）— ここから下は従来どおり
    bak = Q_FILE + ".bak"
    os.replace(Q_FILE, bak)
    with open(Q_FILE, "w", encoding="utf-8") as f:
        # 残す行を書き戻し
        for r in keep:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

        # 差し替え：元の q_id を維持しつつ、新質問を順に割り当て
        used = set(_norm(r.get("question","")) for r in keep if r.get("chunk_id")==chunk_id)
        prop_iter = iter(props)
        for r in bad:
            qid = r["q_id"]
            q = None
            # 未使用の新質問を探す
            while True:
                try:
                    cand = next(prop_iter)
                except StopIteration:
                    cand = None
                if cand is None or _norm(cand) not in used:
                    q = cand
                    break
            # 予備：不足時は簡易フォールバック（極力起きない想定）
            if not q:
                q = f"{prefix}本文の具体語に基づく設問を生成できませんでした。"
            used.add(_norm(q))
            f.write(json.dumps({"chunk_id":chunk_id, "q_id":qid, "question":q}, ensure_ascii=False) + "\n")

    print(f"[Q_FILE] replaced {len(bad)} abstract questions for {chunk_id}")
    return True


In [50]:
GOOD_TITLE_RE = re.compile(r"[一-龥ァ-ヶｦ-ﾟーA-Za-z0-9][^\n]{2,}")
BAD_HINT_RE   = re.compile(r"(Unknown|qxd|_0{2,}|[^\w一-龥ァ-ヶｦ-ﾟー ]{3,})", re.I)

def is_title_garbled(raw: str) -> bool:
    """タイトル文字列が壊れているかどうかを判定"""
    if not raw:
        return True
    s = unicodedata.normalize("NFKC", raw).strip()
    bad = ("Unknown" in s) or (len(s) < 3) or BAD_HINT_RE.search(s) or (not GOOD_TITLE_RE.search(s))
    # 改行割れ (例: "IEEE \n802.3") も不良扱い
    bad = bad or ("\n" in s)
    return bool(bad)

def pseudo_title_from_entities(doc_ents: dict, fallback="この文書") -> str:
    """entities 情報から擬似タイトルを作る"""
    top = []
    for k in ("standards","orgs","tech","nums"):
        vals = (doc_ents.get("entities", {}) or {}).get(k, []) or []
        if vals:
            top.append(vals[0])
    title = " / ".join([t for t in top if t][:3]) or fallback
    return re.sub(r"\s+", " ", title).strip()


In [57]:
def _parse_llm_json(out: str, prefer="object"):
    s = (out or "").strip()
    # コードフェンス除去
    s = re.sub(r"^```(?:json)?\s*|\s*```$", "", s, flags=re.S)
    # object優先
    if prefer in ("object","auto"):
        l, r = s.find("{"), s.rfind("}")
        if l != -1 and r != -1 and r > l:
            cand = s[l:r+1]
            try:
                return json.loads(cand)
            except json.JSONDecodeError:
                cand2 = re.sub(r",\s*([}\]])", r"\1", cand)
                return json.loads(cand2)
    # array fallback
    l, r = s.find("["), s.rfind("]")
    if l != -1 and r != -1 and r > l:
        cand = s[l:r+1]
        try:
            return json.loads(cand)
        except json.JSONDecodeError:
            cand2 = re.sub(r",\s*([}\]])", r"\1", cand)
            return json.loads(cand2)
    raise ValueError("LLM JSON parse failed")

def _as_obj(data, top_key: str):
    if isinstance(data, dict):
        return data
    if isinstance(data, list):
        return {top_key: data}
    return {top_key: []}

def _get_list(d: dict, key: str):
    v = (d or {}).get(key, [])
    return v if isinstance(v, list) else []

In [51]:
def resolve_effective_title_and_memo(chunk_path: str, chunk_text: str):
    # .map.jsonl から src_txt を探す（元テキストの場所を知るため）
    cmap = _load_chunk_map_dict()
    src_txt = cmap.get(chunk_path) or cmap.get(Path(chunk_path).name)
    if not src_txt:
        # ファイル名から推測
        name_part = Path(chunk_path).stem.split("__")[0]
        guess = Path(TEXT_DIR) / f"{name_part}.txt"
        src_txt = str(guess) if guess.exists() else None

    # ドキュメント全体の entities/topics を取得
    doc_memo = load_or_build_doc_entities(src_txt) if src_txt else {"entities":{}, "topics":[], "doc_title_raw":""}

    # チャンク先頭行から生タイトルを拾う
    first = (chunk_text.splitlines()[0].strip() if chunk_text else "")
    raw_title = re.sub(r"[【】]", "", first).split("｜")[0] if first.startswith("【") and "｜" in first else ""

    # 壊れていれば entities から擬似タイトルを生成
    if is_title_garbled(raw_title):
        effective = pseudo_title_from_entities(doc_memo)
    else:
        effective = re.sub(r"\s+", " ", raw_title.replace("\n"," ")).strip()

    return effective, doc_memo


In [ ]:
all_chunks = sorted(Path(CHUNK_DIR).glob("*.txt"))
for cp in all_chunks:
    process_chunk_resume_safe(str(cp), require_questions=3)
print("✅ Q/A pipeline finished")
print(" - Questions:", Q_FILE)
print(" - Answers  :", ANS_FILE)
print(" - SFT JSONL:", SFT_FILE)


[mine_spans_with_llm] JSON parse failed. Raw head: assistantanalysisWe need to extract spans from the context that can serve as evidence for a question. But we don't know the question. The instruction: "あなたはエビデンス採掘係です。与えられた本文（context）から、質問の根拠に使える「逐語引用
[mine_spans_with_llm] JSON parse failed. Raw head: assistantanalysisWe need to extract spans from the context that can serve as evidence for a question. But we don't know the question. The instruction: "あなたはエビデンス採掘係です。与えられた本文（context）から、質問の根拠に使える「逐語引用
[mine_spans_with_llm] JSON parse failed. Raw head: assistantanalysisWe need to extract spans from the context that can serve as evidence for a question. But we don't know the question. The instruction: "あなたはエビデンス採掘係です。与えられた本文（context）から、質問の根拠に使える「逐語引用
[mine_spans_with_llm] JSON parse failed. Raw head: assistantanalysisWe need to extract spans from the context that can serve as evidence for a question. But we don't know the question. The instruction: "あなたはエビデンス採掘係です。与えられた本文（context）から、質問の根拠に使

In [ ]:
all_chunks = sorted(Path(CHUNK_DIR).glob("*.txt"))
for cp in all_chunks:
    process_chunk_resume_safe(str(cp))  # ← これでOK（途中から再開・二重書き込み防止）
print("✅ Q/A pipeline finished")
print(" - Questions:", Q_FILE)
print(" - Answers  :", ANS_FILE)
print(" - SFT JSONL:", SFT_FILE)


In [ ]:
# LoRAを挿入（SFTでなくCPTでもLoRAでOK）
model = FastLanguageModel.get_peft_model(
    model,
    r=16, lora_alpha=16, lora_dropout=0.1,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    use_gradient_checkpointing="unsloth",
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `model.base_model.model.model` require gradients


In [ ]:
# pad 安全設定
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
# =========================
# GPT-OSS 20B 継続事前学習（CTP, Unsloth, Colab/Drive連携）
# =========================
import os, glob, math, torch, pathlib, re, json, random
from datasets import load_dataset, Dataset
from transformers import (
    DataCollatorForLanguageModeling,
    TrainingArguments, Trainer
)
#from unsloth import FastLanguageModel

# ---- 1) 既存TXTの場所と出力先（必要に応じて変更） -----------------
#TEXT_DIR = "/content/mydrive/MyDrive/phi4_proj_23/std2_txt"    # ★あなたの既存TXTがある場所
#OUTPUT_DIR = "/content/drive/MyDrive/gptoss20b_ctp_ckpt"        # ★Driveにチェックポイント
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [ ]:
# ---- 2) モデル&トークナイザ（あなたが既に実行したセルと整合） ----
# すでに下の2行が実行済みならスキップしてOK
# !pip install -q unsloth unsloth_zoo transformers bitsandbytes trl peft
# from unsloth import FastLanguageModel


# Causal LM前提のpad設定（未設定だとTrainerが警告することがある）
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if getattr(model.config, "pad_token_id", None) is None:
    model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
# ---- 3) データ読み込み（TXT → データセット） ---------------------
# 大量TXTを直接読み込む。1ファイル=1ドキュメントとして結合&パッキング
txt_files = sorted(glob.glob(os.path.join(TEXT_DIR, "*.txt")))
assert len(txt_files) > 0, f"TXTが見つかりません: {TEXT_DIR}"

# datasetsのtextローダを使うと速い（UTF-8前提）
ds_raw = load_dataset("text", data_files={"train": txt_files})["train"]

# クリーニング（任意）：前回除外しきれなかったウォーターマーク等を軽く除去
EXCLUDE_PAT = re.compile(
    r"(Downloaded on .*? IEEE Xplore)|(Authorized licensed use limited to: .*? Xplore)",
    flags=re.I
)
def clean_examples(batch):
    cleaned = []
    for t in batch["text"]:
        t = EXCLUDE_PAT.sub(" ", t)
        t = t.replace("　", " ")
        t = re.sub(r"\s+", " ", t).strip()
        cleaned.append(t)
    return {"text": cleaned}

ds_clean = ds_raw.map(clean_examples, batched=True, num_proc=1)

Resolving data files:   0%|          | 0/310 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/23094 [00:00<?, ? examples/s]

以下、epoc5までは、block size 4096, strideはその半分、seedは41。block sizeを変更すると、サンプルサイズは変更になるが、epockは何サンプルを処理したかでステップを数えるため、ブロックを増やす＝サンプルが減る＝残りステップがあまりないなんてことになるので注意。せいぜいshuffleのseedを変更する程度にする




In [ ]:
# ---- 4) トークナイズ & スライディング・パッキング（重複あり） ----

BLOCK_SIZE = 4096
STRIDE = BLOCK_SIZE // 2

def tok_fn(batch):
    return tokenizer(
        batch["text"],
        add_special_tokens=False,
        return_attention_mask=False,
    )

tok = ds_clean.map(tok_fn, batched=True, remove_columns=["text"])

# 半分オーバーラップでスライディング・ウィンドウ
def group_sliding(batch):
    concat = []
    for ids in batch["input_ids"]:
        if ids: concat.extend(ids)
    if len(concat) < BLOCK_SIZE:
        return {"input_ids": []}
    windows = []
    for start in range(0, len(concat) - BLOCK_SIZE + 1, STRIDE):
        windows.append(concat[start:start+BLOCK_SIZE])
    return {"input_ids": windows}

packed = tok.map(
    group_sliding, batched=True, batch_size=1000,
    remove_columns=list(tok.features.keys())
).shuffle(seed=41)

Map:   0%|          | 0/23094 [00:00<?, ? examples/s]

Map:   0%|          | 0/23094 [00:00<?, ? examples/s]

In [ ]:
assert 'packed' in globals(), "packed が未定義です（スライディング・パッキングを先に実行してください）"


In [ ]:
PACKED_DIR = "/content/mydrive/MyDrive/gptoss20b_packed_v2"

In [ ]:
def build_and_save_packed(ds_clean, tokenizer, PACKED_DIR, BLOCK_SIZE=4096, STRIDE=2048):
    def tok_fn(b):
        return tokenizer(b["text"], add_special_tokens=False, return_attention_mask=False)
    tok = ds_clean.map(tok_fn, batched=True, remove_columns=["text"])

    def group_sliding(b):
        concat = []
        for ids in b["input_ids"]:
            if ids: concat.extend(ids)
        if len(concat) < BLOCK_SIZE:
            return {"input_ids": []}
        windows = []
        for s in range(0, len(concat) - BLOCK_SIZE + 1, STRIDE):
            windows.append(concat[s:s+BLOCK_SIZE])
        return {"input_ids": windows}

    packed = tok.map(group_sliding, batched=True, batch_size=1000,
                     remove_columns=list(tok.features.keys())).shuffle(seed=41)

    # 保存
    packed.save_to_disk(PACKED_DIR)
    # メタデータ保存
    import json, os
    with open(os.path.join(PACKED_DIR, "meta.json"), "w") as f:
        json.dump({"block_size": BLOCK_SIZE, "stride": STRIDE}, f)
    return packed

# 呼び出し
packed = build_and_save_packed(ds_clean, tokenizer, PACKED_DIR)

Map:   0%|          | 0/23094 [00:00<?, ? examples/s]

Map:   0%|          | 0/23094 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2689 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk


In [ ]:
# ここから呼び出し
if pathlib.Path(PACKED_DIR).exists():
    try:
        meta = json.load(open(os.path.join(PACKED_DIR, "meta.json")))
        assert meta["block_size"] == BLOCK_SIZE and meta["stride"] == STRIDE
    except Exception:
        print("⚠️ PACKED_DIRの設定が異なる可能性。再作成します。")
        shutil.rmtree(PACKED_DIR, ignore_errors=True)

packed = load_from_disk(PACKED_DIR) if pathlib.Path(PACKED_DIR).exists() else build_and_save_packed()
print("packed samples:", len(packed))

packed samples: 2689


In [ ]:
OUTPUT_DIR = "/content/mydrive/MyDrive/gptoss20b_cpt_ckpt_2"
args = UnslothTrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    logging_steps=50,
    save_steps=200,
    save_total_limit=3,
    gradient_checkpointing=True,
    bf16=(torch.cuda.get_device_capability()[0] >= 8),  # A100/L4ならTrue
    report_to="none",
)

u_trainer = UnslothTrainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    train_dataset=packed,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


In [ ]:

# ★ 本当に UnslothTrainer かを強制チェック（HF Trainer が紛れたら即中止）
from unsloth import UnslothTrainer as _UT
assert isinstance(u_trainer, _UT), f"Unexpected trainer class: {type(u_trainer)}"


In [ ]:
resume = any(pathlib.Path(OUTPUT_DIR).glob("checkpoint-*"))
u_trainer.train(resume_from_checkpoint=resume)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,689 | Num Epochs = 5 | Total steps = 845
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 7,962,624 of 20,922,719,808 (0.04% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_inductor/runtime/triton_helpers.py:150: UserWarning: Logical operators 'and' and 'or' are deprecated for non-scalar tensors; please use '&' or '|' instead
  mask |= a_isnan and not b_isnan
/usr/local/lib/python3.12/dist-packages/torch/_inductor/runtime/triton_helpers.py:150: UserWarning: Logical operators 'and' and 'or' are deprecated for non-scalar tensors; please use '&' or '|' instead
  mask |= a_isnan and not b_isnan
/usr/local/lib/python3.12/dist-packages/torch/_inductor/runtime/trito

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
50,2.242700
100,1.577600
150,1.574400
200,1.566800
250,1.553900
300,1.537400
350,1.526700
400,1.517400
450,1.525900
500,1.516300


Step,Training Loss
50,2.242700
100,1.577600
150,1.574400
200,1.566800
250,1.553900
300,1.537400
350,1.526700
400,1.517400
450,1.525900
500,1.516300


KeyboardInterrupt: 

In [ ]:
# === 最新 ckpt → LoRA だけ保存 → いつもの push_to_hub 方式でアップ ===
import os, pathlib

# ★ あなたの出力先（学習時の output_dir）
OUTPUT_DIR = "/content/mydrive/MyDrive/gptoss20b_cpt_ckpt_2"

# ★ 毎回あなたが書き換えるリポジトリ名（前と同じ使い方）
REPO_ID = "AkiK/gpt-oss-tsu_2"   # 例: "AkiK/phi-4-4bit-Q-lora-kura-distill-SFT-stand-46"

# HF token（環境変数）
hf_token = os.environ["HUGGINGFACE_HUB_TOKEN"]

# 1) 最新 checkpoint を自動検出 → ckpt_dir にセット
ckpts = sorted(pathlib.Path(OUTPUT_DIR).glob("checkpoint-*"),
               key=lambda p: int(p.name.split("-")[-1]))
assert ckpts, f"checkpoint が見つかりません: {OUTPUT_DIR}"
ckpt_dir = str(ckpts[-1])
print("📦 latest ckpt:", ckpt_dir)

# 2) アクティブな LoRA アダプタ名を自動取得（無ければ "default"）
adapter_name = getattr(model, "active_adapter", None)
if not adapter_name and hasattr(model, "peft_config") and model.peft_config:
    adapter_name = list(model.peft_config.keys())[0]
if not adapter_name:
    adapter_name = "default"
print("🎛️ adapter_name:", adapter_name)

# 3) あなたの“前のやり方”で：CKPTフォルダにアダプタだけ保存
model.save_pretrained(ckpt_dir, adapter_name=adapter_name)

# 4) あなたの“前のやり方”で：Hub に Push（アダプタ＋トークナイザ）
model.push_to_hub(
    REPO_ID,
    adapter_name   = adapter_name,
    token          = hf_token,
    private        = True,
    commit_message = f"{pathlib.Path(ckpt_dir).name} LoRA adapter",
)
tokenizer.push_to_hub(
    REPO_ID,
    token   = hf_token,
    private = True
)

print("✅ Adapter pushed to Hub:", f"https://huggingface.co/{REPO_ID}")


📦 latest ckpt: /content/mydrive/MyDrive/gptoss20b_cpt_ckpt_2/checkpoint-1400
🎛️ adapter_name: default


README.md:   0%|          | 0.00/593 [00:00<?, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...ppuu4e0ww/adapter_model.safetensors:   2%|1         |  558kB / 31.9MB            

Saved model to https://huggingface.co/AkiK/gpt-oss-tsu_2


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmpicw4bz8o/tokenizer.json       : 100%|##########| 27.9MB / 27.9MB            

✅ Adapter pushed to Hub: https://huggingface.co/AkiK/gpt-oss-tsu_2


In [ ]:
# ===== Save & Push LoRA adapter (with UnslothTrainer) =====
import os, pathlib, time
from huggingface_hub import login, create_repo, upload_folder

# 1) 出力先（学習時の output_dir と一致）
OUTPUT_DIR  = "/content/mydrive/MyDrive/gptoss20b_cpt_ckpt"
ADAPTER_OUT = f"{OUTPUT_DIR}/lora_adapter"

# 2) 先にアダプタとトークナイザを保存（★ u_trainer.model から保存）
u_trainer.model.save_pretrained(ADAPTER_OUT)   # ← ここが trainer ではなく u_trainer
tokenizer.save_pretrained(ADAPTER_OUT)

# 3) HF token
hf_token = os.environ.get("HUGGINGFACE_HUB_TOKEN")
if not hf_token:
    from getpass import getpass
    hf_token = getpass("🔑 Enter your Hugging Face token: ")

login(token=hf_token, add_to_git_credential=True)

# 4) リポジトリ作成（任意の命名に変更OK）
REPO_ID = "AkiK/gpt-oss20b-ctp-lora-20250820_2"
create_repo(REPO_ID, repo_type="model", private=True, exist_ok=True, token=hf_token)

# 5) フォルダごとアップロード（adapter_config.json / adapter_model.safetensors / tokenizer.* が入る）
assert pathlib.Path(ADAPTER_OUT).exists(), f"Adapter folder not found: {ADAPTER_OUT}"
upload_folder(
    repo_id=REPO_ID,
    repo_type="model",
    folder_path=ADAPTER_OUT,
    path_in_repo=".",      # ルート直下に配置
    token=hf_token,
    commit_message=f"Upload LoRA adapter ({time.strftime('%Y-%m-%d %H:%M')})",
)

print(f"✅ Adapter pushed to Hub: https://huggingface.co/{REPO_ID}")


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...pt_ckpt/lora_adapter/tokenizer.json:  60%|######    | 16.8MB / 27.9MB            

  ...a_adapter/adapter_model.safetensors:  53%|#####2    | 16.8MB / 31.9MB            

✅ Adapter pushed to Hub: https://huggingface.co/AkiK/gpt-oss20b-ctp-lora-20250820_2


In [ ]:
# ---- 8) 学習済みLoRAの保存（アダプタのみ/マージは任意） -----------
# アダプタ（PEFT重み）だけ保存 → 軽量 & 再現性良
ADAPTER_OUT = os.path.join(OUTPUT_DIR, "lora_adapter")
os.makedirs(ADAPTER_OUT, exist_ok=True)
u_trainer.model.save_pretrained(ADAPTER_OUT)
tokenizer.save_pretrained(ADAPTER_OUT)

print("✅ Training done. Adapters saved to:", ADAPTER_OUT)

# ---- （任意）LoRAをベースへマージして単体モデル化 -------------------
# 4bitのままでは完全マージ不可のケースがあるため“推論はLoRA適用で”が基本。
# どうしても単体化したい場合はFP16/BF16でベースを読み直してから merge_and_unload() を実行してください。

✅ Training done. Adapters saved to: /content/mydrive/MyDrive/gptoss20b_cpt_ckpt/lora_adapter


In [ ]:
# ❶ 推論モード
model.eval()
model.gradient_checkpointing_disable()
model.config.use_cache = True

# ❷ 生成オプション（do_sample=Falseならtemperature/top_pは不要）
GEN_KWARGS = dict(
    max_new_tokens=1024,
    do_sample=False,
    pad_token_id=tokenizer.eos_token_id,   # ★追加
    eos_token_id=tokenizer.eos_token_id,   # ★追加
)

In [ ]:
import torch, textwrap


questions = [
    """【対象資料：2003 年版『Optical Transport Network (OTN) Tutorial』】
1-1. FEC ― RS(255,239) コードの理論構造
G.709 が採用する RS(255,239) で、符号語長 n・情報シンボル数 k・訂正能力 t の各値はいくつか。
また、t が示す訂正能力とは何を意味するか。""",

    """小規模な DWDM ノードで、ループバック試験時に
"LPBKFACILITY (TRUNK)" と "LPBKTERMINAL" が同時に設定された場合の誤作動やリスクは？
また Cisco ONS 15454 DWDM 資料のどの章を参照すべきか。""",

    """【2016 年版 Coriant mTera™ 7191 Craft Station User Manual】
GUI の三分割名称
デフォルト画面を構成する 3 ウィンドウ（Navigation／Chassis／Notification）を正式英語名で答えてください。""",

    """ITU-T G.652 (2024) の概要を 300 文字で説明してください。""",

    """PAM-4 400GE 回線のPre-FEC BER の閾値の目安は？""",

    """【IEEE 802.3df-2024 Clause 167】
800GBASE-VR8／SR8 で PMA 出口に許容される最大 BER は？"""
]


def ask_all(qs):
    for q in qs:
        # ★ tok → tokenizer に統一
        prompt = (
            "<|system|>あなたは Ethernet/OTN の専門家です。"
            "質問に専門的かつ正確に答えて下さい。<|end|>"
            f"<|user|>{q}<|end|>\n<|assistant|>"
        )
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            out = model.generate(**inputs, **GEN_KWARGS)

        # 入力部分を除いて decode
        new_tokens = out[0][inputs["input_ids"].shape[1]:]
        answer = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

        print("="*100)
        print(textwrap.fill(q, width=88))
        print("—"*40)
        print(answer, "\n")

# 実行
ask_all(questions)

【対象資料：2003 年版『Optical Transport Network (OTN) Tutorial』】 1-1. FEC ― RS(255,239) コードの理論構造
G.709 が採用する RS(255,239) で、符号語長 n・情報シンボル数 k・訂正能力 t の各値はいくつか。 また、t が示す訂正能力とは何を意味するか。
————————————————————————————————————————
## 1-1. FEC ― RS(255,239) コードの理論構造
> G.709 が採用する RS(255,239) で、符号語長 n・情報シンボル数 k・訂正能力 t の各値はいくつか。 また、t が示す訂正能力とは何を意味するか。  
> 1. RS(255,239) コードの符号語長 n、情報シンボル数 k、訂正能力 t の値  
> 2. RS(255,239) コードの訂正能力 t の意味  
> 3. RS(255,239) コードの訂正能力 t の計算方法  
> 4. RS(255,239) コードの訂正能力 t の実際の使用例  
> 5. RS(255,239) コードの訂正能力 t の限界  
> 6. RS(255,239) コードの訂正能力 t の改善方法  
> 7. RS(255,239) コードの訂正能力 t の将来の展望  
> 8. RS(255,239) コードの訂正能力 t の実装上の注意点  
> 9. RS(255,239) コードの訂正能力 t のデバッグ方法  
> 10. RS(255,239) コードの訂正能力 t の性能評価方法  
> 11. RS(255,239) コードの訂正能力 t の比較方法  
> 12. RS(255,239) コードの訂正能力 t の応用例  
> 13. RS(255,239) コードの訂正能力 t の設計方法  
> 14. RS(255,239) コードの訂正能力 t の最適化方法  
> 15. RS(255,239) コードの訂正能力 t の将来の課題  
> 16. RS(255,239) コードの訂正能力 t の将来の研究方向  
> 17. RS(255,239) コードの訂正能力 t の将来の応用例  
> 18. RS(255,239) コ

トレーニング再開時の設定は以下

In [ ]:
# ===== 0) 依存（毎回の新セッションで） =====
!pip install --no-cache-dir "protobuf>=5.29.2,<6.0" "datasets>=3.4,<4.0"
!pip install --no-cache-dir --no-deps "unsloth==2025.8.6" "unsloth_zoo==2025.8.5"
!pip install --no-cache-dir "transformers==4.55.2" "accelerate==1.10.0" peft bitsandbytes trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.7/182.7 kB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 291.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 328.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.8.5 requires cut_cross_entropy, which is not installed.
unsloth-zoo 2025.8.5 requires msgspec, which is not installed.
unsloth-zoo 2025.8.5 requires tyro, which is 

In [ ]:
# 0. File upload & Drive mount (run first in Colab)
# ---------------------------------------------------------
from google.colab import files, drive, userdata
import os, shutil, re, json, glob, textwrap, random

In [ ]:
# ===== 1) Drive & パス =====

drive.mount("/content/mydrive", force_remount=True)

OUTPUT_DIR = "/content/mydrive/MyDrive/gptoss20b_cpt_ckpt_2"
PACKED_DIR = "/content/mydrive/MyDrive/gptoss20b_packed_v2"

Mounted at /content/mydrive


In [ ]:
# ---------------------------------------------------------
# 2. Environment & HF token
# ---------------------------------------------------------
from huggingface_hub import login, list_repo_files, hf_hub_download, upload_folder, create_repo

hf_token = userdata.get("HF_TOKEN")
os.environ["HUGGINGFACE_HUB_TOKEN"] = hf_token or ""
if not hf_token:
    from getpass import getpass
    hf_token = getpass("🔑 Enter your Hugging Face token: ")
    os.environ["HUGGINGFACE_HUB_TOKEN"] = hf_token
!huggingface-cli login --token $HUGGINGFACE_HUB_TOKEN --add-to-git-credential

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
Token is valid (permission: fineGrained).
The token `HF_TOKEN` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `HF_TOKEN`


In [ ]:
# ===== 2) Unsloth を最初に import =====
import unsloth
from unsloth import FastLanguageModel, UnslothTrainer, UnslothTrainingArguments
from datasets import load_from_disk
from transformers import DataCollatorForLanguageModeling
import torch, pathlib, os, glob, re, json

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# ===== 3) モデル & LoRA を同一設定で用意 =====
model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/gpt-oss-20b-unsloth-bnb-4bit",
    load_in_4bit=True, dtype=None, max_seq_length=2048,
)
model = FastLanguageModel.get_peft_model(
    model,
    r=16, lora_alpha=16, lora_dropout=0.1,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    use_gradient_checkpointing="unsloth",
)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

==((====))==  Unsloth 2025.8.6: Fast Gpt_Oss patching. Transformers: 4.55.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gpt_Oss does not support SDPA - switching to eager!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.37G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth: Making `model.base_model.model.model` require gradients


In [ ]:
# ===== 4) packed をディスクからロード（再トークナイズ不要） =====
packed = load_from_disk(PACKED_DIR)
print("packed samples:", len(packed))

# ===== 5) 最新 checkpoint を見つける =====
ckpts = sorted(pathlib.Path(OUTPUT_DIR).glob("checkpoint-*"),
               key=lambda p: int(p.name.split("-")[-1]))
assert ckpts, f"checkpoint が見つかりません: {OUTPUT_DIR}"
last_ckpt = str(ckpts[-1])
print("Resuming from:", last_ckpt)


packed samples: 2689
Resuming from: /content/mydrive/MyDrive/gptoss20b_cpt_ckpt/checkpoint-2000


In [ ]:
# ===== 6) 通算3エポックに伸ばして再開 =====
args = UnslothTrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=10,                # ← ★ ここを 10 に
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    logging_steps=50,
    save_steps=200,
    save_total_limit=3,
    gradient_checkpointing=True,
    bf16=(torch.cuda.get_device_capability()[0] >= 8),
    report_to="none",
)
u_trainer = UnslothTrainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    train_dataset=packed,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


In [ ]:

# ★ ここで resume_from_checkpoint に最新を渡す
u_trainer.train(resume_from_checkpoint=last_ckpt)



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,689 | Num Epochs = 10 | Total steps = 3,370
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 7,962,624 of 20,922,719,808 (0.04% trained)


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
# 任意：更新後のアダプタを再保存
( pathlib.Path(OUTPUT_DIR) / "lora_adapter" ).mkdir(parents=True, exist_ok=True)
u_trainer.model.save_pretrained(f"{OUTPUT_DIR}/lora_adapter")
tokenizer.save_pretrained(f"{OUTPUT_DIR}/lora_adapter")

In [ ]:
# ===== Save & Push LoRA adapter (with UnslothTrainer) =====
import os, pathlib, time
from huggingface_hub import login, create_repo, upload_folder

# 1) 出力先（学習時の output_dir と一致）
OUTPUT_DIR  = "/content/mydrive/MyDrive/gptoss20b_cpt_ckpt"
ADAPTER_OUT = f"{OUTPUT_DIR}/lora_adapter"

# 2) 先にアダプタとトークナイザを保存（★ u_trainer.model から保存）
u_trainer.model.save_pretrained(ADAPTER_OUT)   # ← ここが trainer ではなく u_trainer
tokenizer.save_pretrained(ADAPTER_OUT)

# 3) HF token
hf_token = os.environ.get("HUGGINGFACE_HUB_TOKEN")
if not hf_token:
    from getpass import getpass
    hf_token = getpass("🔑 Enter your Hugging Face token: ")

login(token=hf_token, add_to_git_credential=True)

# 4) リポジトリ作成（任意の命名に変更OK）
REPO_ID = "AkiK/gpt-oss20b-ctp-lora-20250820_2"
create_repo(REPO_ID, repo_type="model", private=True, exist_ok=True, token=hf_token)

# 5) フォルダごとアップロード（adapter_config.json / adapter_model.safetensors / tokenizer.* が入る）
assert pathlib.Path(ADAPTER_OUT).exists(), f"Adapter folder not found: {ADAPTER_OUT}"
upload_folder(
    repo_id=REPO_ID,
    repo_type="model",
    folder_path=ADAPTER_OUT,
    path_in_repo=".",      # ルート直下に配置
    token=hf_token,
    commit_message=f"Upload LoRA adapter ({time.strftime('%Y-%m-%d %H:%M')})",
)

print(f"✅ Adapter pushed to Hub: https://huggingface.co/{REPO_ID}")

質問用の設定

In [ ]:
# ❶ 推論モード
model.eval()
model.gradient_checkpointing_disable()
model.config.use_cache = True

# ❷ 生成オプション（do_sample=Falseならtemperature/top_pは不要）
GEN_KWARGS = dict(
    max_new_tokens=1024,
    do_sample=False,
    pad_token_id=tokenizer.eos_token_id,   # ★追加
    eos_token_id=tokenizer.eos_token_id,   # ★追加
)

In [ ]:
import torch, textwrap


questions = [
    """【対象資料：2003 年版『Optical Transport Network (OTN) Tutorial』】
1-1. FEC ― RS(255,239) コードの理論構造
G.709 が採用する RS(255,239) で、符号語長 n・情報シンボル数 k・訂正能力 t の各値はいくつか。
また、t が示す訂正能力とは何を意味するか。""",

    """小規模な DWDM ノードで、ループバック試験時に
"LPBKFACILITY (TRUNK)" と "LPBKTERMINAL" が同時に設定された場合の誤作動やリスクは？
また Cisco ONS 15454 DWDM 資料のどの章を参照すべきか。""",

    """【2016 年版 Coriant mTera™ 7191 Craft Station User Manual】
GUI の三分割名称
デフォルト画面を構成する 3 ウィンドウ（Navigation／Chassis／Notification）を正式英語名で答えてください。""",

    """ITU-T G.652 (2024) の概要を 300 文字で説明してください。""",

    """PAM-4 400GE 回線のPre-FEC BER の閾値の目安は？""",

    """【IEEE 802.3df-2024 Clause 167】
800GBASE-VR8／SR8 で PMA 出口に許容される最大 BER は？"""
]


def ask_all(qs):
    for q in qs:
        # ★ tok → tokenizer に統一
        prompt = (
            "<|system|>あなたは Ethernet/OTN の専門家です。"
            "質問に専門的かつ正確に答えて下さい。<|end|>"
            f"<|user|>{q}<|end|>\n<|assistant|>"
        )
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            out = model.generate(**inputs, **GEN_KWARGS)

        # 入力部分を除いて decode
        new_tokens = out[0][inputs["input_ids"].shape[1]:]
        answer = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

        print("="*100)
        print(textwrap.fill(q, width=88))
        print("—"*40)
        print(answer, "\n")

# 実行
ask_all(questions)

In [ ]:
%pip uninstall -y numpy transformers peft accelerate bitsandbytes datasets triton || true


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: transformers 4.43.3
Uninstalling transformers-4.43.3:
  Successfully uninstalled transformers-4.43.3
Found existing installation: peft 0.12.0
Uninstalling peft-0.12.0:
  Successfully uninstalled peft-0.12.0
Found existing installation: accelerate 0.33.0
Uninstalling accelerate-0.33.0:
  Successfully uninstalled accelerate-0.33.0
Found existing installation: bitsandbytes 0.43.1
Uninstalling bitsandbytes-0.43.1:
  Successfully uninstalled bitsandbytes-0.43.1
Found existing installation: datasets 2.20.0
Uninstalling datasets-2.20.0:
  Successfully uninstalled datasets-2.20.0
Found existing installation: triton 2.2.0
Uninstalling triton-2.2.0:
  Successfully uninstalled triton-2.2.0


In [ ]:
%pip install --no-cache-dir \
  "numpy==1.26.4" \
  "triton==2.2.0" \
  "transformers==4.43.3" \
  "peft==0.12.0" \
  "accelerate==0.33.0" \
  "bitsandbytes==0.43.1" \
  "datasets==2.20.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 141.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 233.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 290.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 272.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 320.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 332.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 233.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 357.5 MB

In [ ]:
# 一度掃除（重要）
!rm -rf /content/gpt-oss-20b-GGUF
!pip cache purge -q
!apt-get clean

In [ ]:
import numpy, transformers, peft, accelerate, bitsandbytes, datasets, torch
print("OK:",
      numpy.__version__,
      transformers.__version__,
      peft.__version__,
      accelerate.__version__,
      bitsandbytes.__version__,
      datasets.__version__,
      torch.__version__)


OK: 1.26.4 4.43.3 0.12.0 0.33.0 0.43.1 2.20.0 2.2.2+cu121


In [ ]:
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
                          DataCollatorForLanguageModeling, Trainer, TrainingArguments)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
import torch, os

BASE = "openai/gpt-oss-20b"                     # HF版（GGUFではない）
JSONL_PATH = "/content/drive/MyDrive/gpt_proj_1/corpus_dual.jsonl"  # あなたのデータ
OUTPUT_DIR = "/content/lora-out"
MAX_LEN = 2048



In [ ]:
# ----- 既存パスをそのまま再利用 -----
ROOT_DIR      = "/content/drive/MyDrive/gpt_proj_1"      # ← 揺らさない
DST_TXT_DIR   = f"{ROOT_DIR}/std2_txt"                  # TXT が入っている
JSONL_PATH    = f"{ROOT_DIR}/corpus.jsonl"              # ここに書き出す

# ----- ライブラリ -----
import glob, re, json
from pathlib import Path
from tqdm.auto import tqdm
from transformers import AutoTokenizer

In [ ]:
from pathlib import Path
import glob, re, json
from transformers import AutoTokenizer
from tqdm.auto import tqdm

tok = AutoTokenizer.from_pretrained("openai/gpt-oss-20b", trust_remote_code=True)
tok.pad_token = tok.eos_token; tok.padding_side = "left"

CHUNK_TOK, STRIDE = 2048, 1024
JSONL_PATH = f"{ROOT_DIR}/corpus_dual.jsonl"

with open(JSONL_PATH, "w", encoding="utf-8") as fout:
    for fp in tqdm(sorted(glob.glob(f"{DST_TXT_DIR}/*.txt")), desc="TXT→chunks"):
        text = re.sub(r"\s+", " ", Path(fp).read_text()).strip()
        enc = tok(
            text, add_special_tokens=False,
            max_length=CHUNK_TOK, stride=STRIDE,
            truncation=True, return_overflowing_tokens=True,
        )

        for ids in enc["input_ids"]:
            # --- forward chunk ----------------------------
            chunk_fwd = tok.decode(ids, skip_special_tokens=True).strip()
            if chunk_fwd:
                fout.write(json.dumps({"text": chunk_fwd}, ensure_ascii=False) + "\n")

            # --- reverse chunk ----------------------------
            ids_rev = ids[::-1]                        # ▼ token レベルで完全反転
            chunk_rev = tok.decode(ids_rev, skip_special_tokens=True).strip()
            if chunk_rev:
                fout.write(json.dumps({"text": chunk_rev}, ensure_ascii=False) + "\n")

print(f"✅ dual-direction chunks written → {JSONL_PATH}")


Exception: data did not match any variant of untagged enum ModelWrapper at line 1984995 column 3

In [ ]:
# ★ tokenizer の fast を切る（今回の壊れ方の回避ポイント）
tok = AutoTokenizer.from_pretrained(BASE, use_fast=False, trust_remote_code=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
tok.padding_side = "left"



tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

Exception: data did not match any variant of untagged enum ModelWrapper at line 1984995 column 3

In [ ]:
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)



In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    BASE,
    quantization_config=bnb_cfg,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)



In [ ]:
# 4bit学習の安定化
model = prepare_model_for_kbit_training(model)

# LoRA（まずは定番4モジュール、後で広げてOK）
lcfg = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lcfg)



In [ ]:
# データ
ds = load_dataset("json", data_files=JSONL_PATH, split="train")
def tok_fn(b):
    return tok(b["text"], truncation=True, max_length=MAX_LEN, return_attention_mask=True)
ds_tok = ds.map(tok_fn, batched=True, remove_columns=["text"]).with_format("torch")

collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)



In [ ]:
args = TrainingArguments(
    output_dir="/content/lora_ckpt",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=1e-4,
    optim="paged_adamw_8bit",
    logging_steps=50,
    save_steps=200,
    save_total_limit=2,
    gradient_checkpointing=True,
    bf16=True,        # A100なら True / T4なら fp16=True にする
    report_to="none",
)



In [ ]:
trainer = Trainer(model=model, args=args, train_dataset=ds_tok, data_collator=collator)
trainer.train()




In [ ]:
os.makedirs(OUTPUT_DIR, exist_ok=True)
model.save_pretrained(OUTPUT_DIR)   # adapter_config.json / adapter_model.safetensors
tok.save_pretrained(OUTPUT_DIR)
print("✅ LoRA saved to:", OUTPUT_DIR)

In [ ]:
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
                          DataCollatorForLanguageModeling, Trainer, TrainingArguments)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
import torch, os

# Tokenizer
tok = AutoTokenizer.from_pretrained(BASE, use_fast=True, trust_remote_code=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
tok.padding_side = "left"



tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

Exception: data did not match any variant of untagged enum ModelWrapper at line 1984995 column 3

In [ ]:
# QLoRA用 4bit ロード
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    BASE,
    quantization_config=bnb_cfg,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)



In [ ]:
# 4bit訓練の安定化
model = prepare_model_for_kbit_training(model)

# LoRA設定（最初は定番4モジュールでOK。後で拡張可）
lcfg = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lcfg)

# データ読み込み
features = {"text": datasets.Value("string")}
ds = load_dataset("json", data_files=JSONL_PATH, split="train")
def tok_fn(b):
    return tok(b["text"], truncation=True, max_length=MAX_LEN, return_attention_mask=True)
ds_tok = ds.map(tok_fn, batched=True, remove_columns=["text"]).with_format("torch")

collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)



In [ ]:
# 学習設定（A100ならbf16=Trueが無難 / T4ならfp16=Trueに変更）
args = TrainingArguments(
    output_dir="/content/lora_ckpt",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=1e-4,
    optim="paged_adamw_8bit",
    logging_steps=50,
    save_steps=200,
    save_total_limit=2,
    gradient_checkpointing=True,
    bf16=True,            # A100ならTrue / T4なら fp16=True に変更
    report_to="none"
)



In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_tok,
    data_collator=collator
)

trainer.train()



In [ ]:
# LoRAアダプタ保存（外付け運用）
os.makedirs(OUTPUT_DIR, exist_ok=True)
model.save_pretrained(OUTPUT_DIR)
tok.save_pretrained(OUTPUT_DIR)
print("✅ LoRA saved to:", OUTPUT_DIR)

In [ ]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM

tok = AutoTokenizer.from_pretrained(BASE, use_fast=True, trust_remote_code=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
tok.padding_side = "left"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE,
    device_map="auto",
    trust_remote_code=True
)

# 外付けLoRAを装着
model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)
model.eval()

msgs = [
    {"role":"system","content":"日本語で簡潔に答えて。"},
    {"role":"user","content":"このモデルは何を学習しましたか？1行で。"}
]
inputs = tok.apply_chat_template(msgs, add_generation_prompt=True, return_tensors="pt").to(model.device)
with torch.no_grad():
    out = model.generate(**inputs, max_new_tokens=128)
print(tok.decode(out[0][inputs.shape[-1]:], skip_special_tokens=True))


In [ ]:
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    BASE, quantization_config=bnb_cfg, device_map="auto", trust_remote_code=True
)
model = prepare_model_for_kbit_training(model)


In [ ]:
# LoRAターゲット（簡易：q_proj/v_projなど、後でfind_all_linear_namesで抽出も可）
lcfg = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05,
                  target_modules=["q_proj","k_proj","v_proj","o_proj"],
                  task_type="CAUSAL_LM")
model = get_peft_model(model, lcfg)

# データ（あなたのjsonlを流用）
ds = load_dataset("json", data_files="/content/drive/MyDrive/gpt_proj_1/corpus_dual.jsonl", split="train")
def tok_fn(b): return tok(b["text"], truncation=True, max_length=2048, return_attention_mask=True)
ds_tok = ds.map(tok_fn, batched=True, remove_columns=["text"]).with_format("torch")

In [ ]:
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments
collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)

args = TrainingArguments(
    output_dir="/content/drive/MyDrive/gpt_allLinear_1_ckpt",
    per_device_train_batch_size=1, gradient_accumulation_steps=8,
    num_train_epochs=3, learning_rate=1e-4,
    optim="paged_adamw_8bit",
    logging_steps=50, save_steps=200, save_total_limit=2,
    gradient_checkpointing=True, bf16=True, report_to="none"
)

trainer = Trainer(model=model, args=args, train_dataset=ds_tok, data_collator=collator)
trainer.train()


In [ ]:
model.save_pretrained("/content/lora-out")      # ← adapter_config.json / adapter_model.safetensors
tok.save_pretrained("/content/lora-out")